# Shenzhen exposure-weight demo (Colab / local)

This notebook computes **user × spatial-unit exposure weights** (grid or H3 hex) in lunch/dinner windows using `poi_visit_aggregator`.

Outputs (per city):
- grid: `user_grid_time_strict_filled_<city>.parquet` + `qa_summary_strict_filled_<city>.csv`
- hex: `user_hex_time_strict_filled_<city>.parquet` + `qa_summary_hex_strict_filled_<city>.csv`

Two common workflows:
1) **Visualize existing outputs (recommended)**: set `EXPO_DIR` to the folder that contains the two output files.
2) **Run export (slow)**: set `RUN_EXPORT=True` and configure `STAYPOINTS/UUID_TABLE` plus `GRID_META` (grid) or `HEX_META` (hex).

Tip: for stability/performance, put `tmp_root` and DuckDB temp on a local disk (Colab: `/tmp`, Windows: e.g. `C:\\temp`).


In [2]:
from __future__ import annotations

from pathlib import Path
import os
import sys

import numpy as np
import pandas as pd

# --- Colab detection ---
try:
    from google.colab import drive  # type: ignore

    IN_COLAB = True
except Exception:
    IN_COLAB = False


def _find_repo_root(start: Path) -> Path:
    p = start.resolve()
    for parent in [p, *p.parents]:
        if (parent / "pyproject.toml").exists() and (parent / "poi_visit_aggregator").exists():
            return parent
    return start.resolve()


# --- Colab bootstrap (Drive + optional clone) ---
# If you open this notebook directly in Colab (not opened from the repo),
# keep CLONE_REPO_IN_COLAB=True to clone the repo into Drive and add it to sys.path.
CLONE_REPO_IN_COLAB = True
COLAB_TARGET_DIR = Path("/content/drive/MyDrive/Script/Module")
COLAB_REPO_PATH = COLAB_TARGET_DIR / "poi_visit_aggregator"

if IN_COLAB and CLONE_REPO_IN_COLAB:
    drive.mount("/content/drive")
    COLAB_TARGET_DIR.mkdir(parents=True, exist_ok=True)
    os.chdir(str(COLAB_TARGET_DIR))

    if not COLAB_REPO_PATH.exists():
        get_ipython().system("git clone https://github.com/weipengdeng/poi_visit_aggregator.git")
    else:
        try:
            get_ipython().system(f"git -C {COLAB_REPO_PATH} pull")
        except Exception:
            pass

    if str(COLAB_REPO_PATH) in sys.path:
        sys.path.remove(str(COLAB_REPO_PATH))
    sys.path.insert(0, str(COLAB_REPO_PATH))
    os.chdir(str(COLAB_REPO_PATH))

if not IN_COLAB:
    # VSCode/Jupyter sometimes sets CWD to `notebooks/`. Make repo imports work either way.
    REPO_ROOT = _find_repo_root(Path.cwd())
    os.chdir(str(REPO_ROOT))
    if str(REPO_ROOT) not in sys.path:
        sys.path.insert(0, str(REPO_ROOT))

try:
    import psutil  # type: ignore
except Exception:
    psutil = None


def mem_gb():
    """Return (rss_gb, avail_gb, used_pct)."""
    if psutil is None:
        return (np.nan, np.nan, np.nan)
    vm = psutil.virtual_memory()
    rss = psutil.Process(os.getpid()).memory_info().rss
    return (float(rss) / 1e9, float(vm.available) / 1e9, float(vm.percent))


pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

print("IN_COLAB:", IN_COLAB)
print("CWD:", Path.cwd())
rss_gb, avail_gb, used_pct = mem_gb()
print(f"Memory: RSS={rss_gb:.2f} GB | avail={avail_gb:.2f} GB | used={used_pct:.0f}%")


IN_COLAB: False
CWD: E:\OneDrive\Git\poi_visit_aggregator
Memory: RSS=0.16 GB | avail=28.84 GB | used=58%


In [3]:
# Install export deps (run once)
# If you rerun the notebook often, you can comment this cell after the first run.
#
# Colab:
# !pip -q install -e ".[export]"
#
# Local (VSCode/Jupyter):
# !{sys.executable} -m pip install -e ".[export]"
#
# Optional (map viz):
# !{sys.executable} -m pip install plotly


In [4]:
from pathlib import Path
import os

CITY = "Shenzhen"
# City code that matches staypoints `c_code`.
GRID_UID_CODE = "440300"  # used when UNIT_MODE='grid'
HEX_UID_CODE = "440300"   # used when UNIT_MODE='hex'

# Spatial unit: 'grid' (default) or 'hex' (H3).
UNIT_MODE = os.environ.get("POI_VISIT_UNIT_MODE", "grid").strip().lower()
if UNIT_MODE not in {"grid", "hex"}:
    raise ValueError("POI_VISIT_UNIT_MODE must be grid|hex")
UNIT_UID_COL = "grid_uid" if UNIT_MODE == "grid" else "hex_uid"

# If you already have the outputs, keep RUN_EXPORT=False and set EXPO_DIR.
RUN_EXPORT = False

# --- Outputs location (visualize-only) ---
if IN_COLAB:
    DRIVE_ROOT = Path("/content/drive/MyDrive")
    EXPO_DIR = DRIVE_ROOT / "Project/202512_EFE/data/expo"
else:
    EXPO_DIR = Path(os.environ.get("POI_VISIT_EXPO_DIR", r"E:\\OneDrive\\HKU\\Project\\202512_EFE\\data\\expo"))

OUT_CITY_DIR = EXPO_DIR
OUT_FILE = OUT_CITY_DIR / (
    f"user_grid_time_strict_filled_{CITY}.parquet"
    if UNIT_MODE == "grid"
    else f"user_hex_time_strict_filled_{CITY}.parquet"
)
QA_FILE = OUT_CITY_DIR / (
    f"qa_summary_strict_filled_{CITY}.csv" if UNIT_MODE == "grid" else f"qa_summary_hex_strict_filled_{CITY}.csv"
)

# --- Grid meta for map visualization ---
# It should contain `data.center_x/center_y` and `crs` (see `grid_meta_<city>.json`).
GRID_META_VIZ = None
if UNIT_MODE == "grid":
    GRID_META_VIZ = os.environ.get("POI_VISIT_GRID_META_VIZ", "").strip()
    if GRID_META_VIZ:
        GRID_META_VIZ = Path(GRID_META_VIZ)
    else:
        # Best-effort auto-discovery under your project `data/`.
        default_project_root = OUT_CITY_DIR
        if OUT_CITY_DIR.name.lower() == "expo" and OUT_CITY_DIR.parent.name.lower() == "data":
            default_project_root = OUT_CITY_DIR.parent.parent
        PROJECT_ROOT = Path(os.environ.get("POI_VISIT_PROJECT_ROOT", str(default_project_root)))
        candidates = list((PROJECT_ROOT / "data").rglob(f"grid_meta_{CITY}.json")) if (PROJECT_ROOT / "data").exists() else []
        GRID_META_VIZ = candidates[0] if candidates else None

# --- Hex meta (H3) ---
HEX_META = os.environ.get("POI_VISIT_HEX_META", "").strip()
if HEX_META:
    HEX_META = Path(HEX_META)
else:
    default_hex = Path.cwd() / f"hex_meta_{CITY}_{HEX_UID_CODE}.json"
    HEX_META = default_hex if default_hex.exists() else None

# --- (Optional) run export in this notebook ---
if RUN_EXPORT:
    if IN_COLAB:
        DRIVE_ROOT = Path("/content/drive/MyDrive")
        DATA_ROOT = DRIVE_ROOT / "Project/202512_EFE"

        RUN_TAG = "access_k10_popcentroid_v1"
        OUT_ROOT = DATA_ROOT / "data" / "derived" / "accessibility" / RUN_TAG
        GRID_JSON_DIR = OUT_ROOT / "grid_json"

        UUID_TABLE = DATA_ROOT / "data/jike/uuid.csv"  # .csv or .parquet
        STAYPOINTS = [DATA_ROOT / "data/jike/track.csv"]
        GRID_META = GRID_JSON_DIR / f"grid_meta_{CITY}.json"
        HEX_META = GRID_JSON_DIR / f"hex_meta_{CITY}_{HEX_UID_CODE}.json"
        OUT_DIR = OUT_ROOT / "out/poi_visit_aggregator"

        # Recommended: keep temp files off Google Drive.
        TMP_ROOT = Path("/tmp/poi_visit_aggregator_tmp")
        DUCKDB_TEMP_DIR = Path("/tmp/duckdb_tmp")
    else:
        # TODO: edit these paths to your local disk (raw inputs).
        DATA_ROOT = Path(r"D:\\Project\\202512_EFE")
        UUID_TABLE = DATA_ROOT / "uuid.csv"  # .csv or .parquet
        STAYPOINTS = [DATA_ROOT / "staypoints.csv"]
        GRID_META = DATA_ROOT / f"grid_meta_{CITY}.json"
        HEX_META = DATA_ROOT / f"hex_meta_{CITY}_{HEX_UID_CODE}.json"
        OUT_DIR = DATA_ROOT / "out/poi_visit_aggregator"

        # Put temp on a fast local disk (avoid OneDrive folders).
        temp_dir = Path(os.environ.get("TEMP", "."))
        TMP_ROOT = Path(os.environ.get("POI_VISIT_TMP_ROOT", str(temp_dir / "poi_visit_aggregator_tmp")))
        DUCKDB_TEMP_DIR = Path(os.environ.get("POI_VISIT_DUCKDB_TMP", str(temp_dir / "duckdb_tmp")))

    # If staypoints contain nationwide users, keep this on for speed.
    FILTER_CITY_CODE = True
    CITY_CODE_COL = "c_code"
    CITY_CODE_VALUE = GRID_UID_CODE

    OUT_CITY_DIR = OUT_DIR / CITY
    OUT_FILE = OUT_CITY_DIR / (
        f"user_grid_time_strict_filled_{CITY}.parquet"
        if UNIT_MODE == "grid"
        else f"user_hex_time_strict_filled_{CITY}.parquet"
    )
    QA_FILE = OUT_CITY_DIR / (
        f"qa_summary_strict_filled_{CITY}.csv"
        if UNIT_MODE == "grid"
        else f"qa_summary_hex_strict_filled_{CITY}.csv"
    )

print("UNIT_MODE:", UNIT_MODE)
print("UNIT_UID_COL:", UNIT_UID_COL)
print("OUT_FILE:", OUT_FILE)
print("QA_FILE:", QA_FILE)
print("GRID_META_VIZ:", GRID_META_VIZ)
print("HEX_META:", HEX_META)

OUT_CITY_DIR


OUT_FILE: E:\OneDrive\HKU\Project\202512_EFE\data\expo\user_grid_time_strict_filled_Shenzhen.parquet
QA_FILE: E:\OneDrive\HKU\Project\202512_EFE\data\expo\qa_summary_strict_filled_Shenzhen.csv
GRID_META_VIZ: E:\OneDrive\HKU\Project\202512_EFE\data\derived\accessibility\__grid_export_test\grid_json\grid_meta_Shenzhen.json


WindowsPath('E:/OneDrive/HKU/Project/202512_EFE/data/expo')

In [5]:
# Optional: map your column names if they differ.
# Fill in only what you need.
SCHEMA_MAP = {
    "staypoints": {
        # "uuid": "uuid",
        # "start_time": "start_ms",
        # "end_time": "end_ms",
        # one of (x,y) or (lon,lat) or location
        # "lon": "lon",
        # "lat": "lat",
        # "source": "source",
    },
    "uuid_table": {
        # "uuid": "uuid",
    },
}
SCHEMA_MAP


{'staypoints': {}, 'uuid_table': {}}

In [6]:
import os
import time
from contextlib import contextmanager

try:
    import psutil  # type: ignore
except Exception:
    psutil = None


def rss_mb() -> float:
    if psutil is None:
        return float("nan")
    return psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024


@contextmanager
def step(name: str):
    t0 = time.perf_counter()
    m0 = rss_mb()
    print(f"[START] {name} (RAM={m0:,.0f} MB)")
    try:
        yield
    finally:
        dt = time.perf_counter() - t0
        m1 = rss_mb()
        print(f"[ END ] {name} (dt={dt:,.1f}s, RAM={m1:,.0f} MB, Δ={m1 - m0:,.0f} MB)")


In [7]:
RESUME_STAGE2 = False  # set True to reuse existing Stage-1 parts
KEEP_INTERMEDIATE = True  # keep Stage-1 parts (useful for resume/debug)

if RUN_EXPORT:
    if UNIT_MODE == "grid":
        from poi_visit_aggregator.export_user_grid_time_strict_filled import (
            export_user_grid_time_strict_filled,
        )

        with step("export_user_grid_time_strict_filled"):
            export_user_grid_time_strict_filled(
                city=CITY,
                staypoints=[Path(p) for p in STAYPOINTS],
                staypoints_format="auto",  # or csv/parquet
                uuid_table=UUID_TABLE,
                grid_meta_path=GRID_META,
                out_dir=OUT_DIR,
                tmp_root=TMP_ROOT,
                duckdb_temp_dir=DUCKDB_TEMP_DIR,
                schema_map=SCHEMA_MAP,
                output_grid_uid=True,
                output_grid_id=False,
                grid_uid_code=GRID_UID_CODE,
                grid_uid_prefix="grid",
                grid_uid_order="col_row",
                filter_city_code=FILTER_CITY_CODE,
                city_code_col=CITY_CODE_COL,
                city_code_value=CITY_CODE_VALUE,
                windows=["lunch", "dinner"],
                min_interval_minutes=5,
                point_source_filter=True,
                point_source_value="cell_appearance",
                drop_uuid_not_in_table=True,  # skip users not in UUID_TABLE
                timestamps_are_utc=True,
                tz_offset_hours=8,
                epoch_unit="ms",
                coords_already_projected=False,
                uid64_hash_method="xxh64",  # faster if installed; else use sha256_64
                buckets=256,
                batch_size=1_000_000,
                log_every_batches=500,
                overlap_rounding="floor",
                oob_mode="drop",
                threads=max(1, (os.cpu_count() or 8) - 1),
                memory_limit="8GB",
                id_mode="uuid",  # uuid|uid64|both
                resume_stage2=RESUME_STAGE2,
                keep_intermediate=KEEP_INTERMEDIATE,
            )
    else:
        from poi_visit_aggregator.export_user_hex_time_strict_filled import (
            export_user_hex_time_strict_filled,
        )

        if HEX_META is None:
            raise FileNotFoundError(
                "HEX_META not found. Set env var POI_VISIT_HEX_META to your hex_meta_*.json path."
            )
        with step("export_user_hex_time_strict_filled"):
            export_user_hex_time_strict_filled(
                city=CITY,
                staypoints=[Path(p) for p in STAYPOINTS],
                staypoints_format="auto",  # or csv/parquet
                uuid_table=UUID_TABLE,
                hex_meta_path=HEX_META,
                out_dir=OUT_DIR,
                tmp_root=TMP_ROOT,
                duckdb_temp_dir=DUCKDB_TEMP_DIR,
                schema_map=SCHEMA_MAP,
                output_hex_uid=True,
                output_h3_id=True,
                output_h3_int=False,
                hex_uid_code=HEX_UID_CODE,
                hex_uid_prefix="hex",
                filter_city_code=FILTER_CITY_CODE,
                city_code_col=CITY_CODE_COL,
                city_code_value=CITY_CODE_VALUE,
                windows=["lunch", "dinner"],
                min_interval_minutes=5,
                point_source_filter=True,
                point_source_value="cell_appearance",
                drop_uuid_not_in_table=True,  # skip users not in UUID_TABLE
                timestamps_are_utc=True,
                tz_offset_hours=8,
                epoch_unit="ms",
                uid64_hash_method="xxh64",  # faster if installed; else use sha256_64
                buckets=256,
                batch_size=1_000_000,
                log_every_batches=500,
                overlap_rounding="floor",
                oob_mode="drop",
                threads=max(1, (os.cpu_count() or 8) - 1),
                memory_limit="8GB",
                id_mode="uuid",  # uuid|uid64|both
                resume_stage2=RESUME_STAGE2,
                keep_intermediate=KEEP_INTERMEDIATE,
            )
else:
    print("RUN_EXPORT=False: skip export; visualize existing outputs.")

OUT_FILE


RUN_EXPORT=False: skip export; visualize existing outputs.


WindowsPath('E:/OneDrive/HKU/Project/202512_EFE/data/expo/user_grid_time_strict_filled_Shenzhen.parquet')

In [8]:
import os
import duckdb
import pandas as pd

assert OUT_FILE.exists(), f"Missing output parquet: {OUT_FILE}"
assert QA_FILE.exists(), f"Missing QA csv: {QA_FILE}"

qa = pd.read_csv(QA_FILE)
display(qa.T.head(120))

con = duckdb.connect()
con.execute(f"PRAGMA threads={max(1, (os.cpu_count() or 8) - 1)}")

# Light preview (avoid loading the full parquet into memory)
df_head = con.execute("select * from read_parquet(?) limit 5", [str(OUT_FILE)]).fetchdf()
display(df_head)

unit_col = UNIT_UID_COL
overall_sql = f"""
    select
      count(*) as n_rows,
      count(distinct uuid) as n_users,
      count(distinct {unit_col}) as n_units,
      sum(tau_strict_min) as tau_strict_min_sum,
      sum(tau_fill_min) as tau_fill_min_sum,
      sum(tau_filled_min) as tau_filled_min_sum,
      sum(case when tau_fill_min > 0 then 1 else 0 end) as n_rows_tau_fill_pos
    from read_parquet(?)
    """
overall = con.execute(overall_sql, [str(OUT_FILE)]).fetchdf()
display(overall.T)

by = con.execute(
    '''
    select
      "window",
      is_weekend,
      count(*) as n_rows,
      sum(tau_strict_min) as tau_strict_min_sum,
      sum(tau_fill_min) as tau_fill_min_sum,
      sum(tau_filled_min) as tau_filled_min_sum
    from read_parquet(?)
    group by 1,2
    order by 1,2
    ''',
    [str(OUT_FILE)],
).fetchdf()
by["fill_share"] = by["tau_fill_min_sum"] / by["tau_filled_min_sum"]
display(by)


,0
city,Shenzhen
tmp_root,C:/Users/Administrator/AppData/Local/Temp/3/po...
tmp_dir,C:/Users/Administrator/AppData/Local/Temp/3/po...
id_mode,uuid
output_grid_uid,True
...,...
lunch__point_max_gap_p90,150.0
lunch__point_max_gap_p99,150.0
dinner__point_max_gap_p50,240.0
dinner__point_max_gap_p90,240.0


,grid_uid,window,is_weekend,uuid,tau_strict_min,tau_fill_min,tau_filled_min
0,grid_440300_78_25,lunch,False,1431f81b-aae0-4a40-b20e-56db94c69f61,104,0.0,104.0
1,grid_440300_50_34,dinner,False,5777e479-ba9a-4bf2-af79-fa970b8b24ee,460,0.0,460.0
2,grid_440300_58_35,dinner,False,46708846-6690-4830-af07-5756925e73e8,119,0.0,119.0
3,grid_440300_58_38,lunch,True,6e816df6-fd50-4146-b5ce-6264d947265c,300,0.0,300.0
4,grid_440300_39_31,lunch,False,67bc3a8d9cafc4ba9cd885b19f40794a,35,0.0,35.0


,0
n_rows,8111108.0
n_users,696941.0
n_grids,6381.0
tau_strict_min_sum,906362649.0
tau_fill_min_sum,2479.0
tau_filled_min_sum,906365128.0
n_rows_tau_fill_pos,13.0


,window,is_weekend,n_rows,tau_strict_min_sum,tau_fill_min_sum,tau_filled_min_sum,fill_share
0,dinner,False,3235151,405955416.0,1479.0,405956895.0,0.000004
1,dinner,True,1620013,146447562.0,465.0,146448027.0,0.000003
2,lunch,False,2125357,260336955.0,400.0,260337355.0,0.000002
3,lunch,True,1130587,93622716.0,135.0,93622851.0,0.000001


In [9]:
# Grid-level aggregation + map visualization
import json
import os
from pathlib import Path

import numpy as np
import pandas as pd
from pyproj import CRS, Transformer
import plotly.express as px

WINDOW = "all"  # "lunch" | "dinner" | "all"
IS_WEEKEND = "all"  # True | False | "all"
MAX_POINTS_ON_MAP = 4000  # reduce if rendering is slow

where = []
params = [str(OUT_FILE)]
if WINDOW != "all":
    where.append('"window" = ?')
    params.append(WINDOW)
if IS_WEEKEND != "all":
    where.append("is_weekend = ?")
    params.append(IS_WEEKEND)
where_sql = ("where " + " and ".join(where)) if where else ""

grid_agg = con.execute(
    f'''
    select
      grid_uid,
      sum(tau_filled_min) as tau_filled_min_sum,
      sum(tau_strict_min) as tau_strict_min_sum,
      sum(tau_fill_min) as tau_fill_min_sum,
      count(*) as n_user_rows
    from read_parquet(?)
    {where_sql}
    group by 1
    ''',
    params,
).fetchdf()

grid_agg[["col", "row"]] = grid_agg["grid_uid"].str.extract(r".*_(\d+)_(\d+)$").astype(int)
grid_agg["fill_share"] = grid_agg["tau_fill_min_sum"] / grid_agg["tau_filled_min_sum"]
grid_agg["w_log"] = np.log1p(grid_agg["tau_filled_min_sum"])

display(grid_agg.sort_values("tau_filled_min_sum", ascending=False).head(20))

if GRID_META_VIZ is None:
    raise FileNotFoundError(
        "GRID_META_VIZ not found. Set env var POI_VISIT_GRID_META_VIZ to your `grid_meta_<city>.json`."
    )

meta_path = Path(GRID_META_VIZ)
obj = json.loads(meta_path.read_text(encoding="utf-8"))
data = obj.get("data", {})

cell_size_m = float(obj.get("grid_size_m") or obj.get("cell_size_m") or 500.0)
grid_crs = obj.get("crs") or obj.get("grid_crs") or (f"EPSG:{obj.get('epsg')}" if obj.get("epsg") else None)
if grid_crs is None:
    raise ValueError(f"grid_meta missing `crs/epsg`: {meta_path}")

ids = data.get("grid_uid") or data.get("grid_id")
center_x = np.asarray(data.get("center_x", []), dtype=float)
center_y = np.asarray(data.get("center_y", []), dtype=float)
if ids is None or len(ids) == 0 or center_x.size == 0 or center_y.size == 0:
    raise ValueError(f"grid_meta missing `data.grid_uid/grid_id` or `data.center_x/center_y`: {meta_path}")

ids = pd.Series(ids, dtype="string")
parts = ids.str.extract(r".*_(?P<col>\d+)_(?P<row>\d+)$")
col_m = parts["col"].astype(int).to_numpy()
row_m = parts["row"].astype(int).to_numpy()

origin_x = float(np.median(center_x - (col_m + 0.5) * cell_size_m))
origin_y = float(np.median(center_y - (row_m + 0.5) * cell_size_m))

transformer = Transformer.from_crs(CRS.from_user_input(grid_crs), CRS.from_user_input("EPSG:4326"), always_xy=True)
x = origin_x + (grid_agg["col"].to_numpy() + 0.5) * cell_size_m
y = origin_y + (grid_agg["row"].to_numpy() + 0.5) * cell_size_m
lon, lat = transformer.transform(x, y)
grid_agg["lon"] = lon
grid_agg["lat"] = lat

plot_df = grid_agg.sort_values("tau_filled_min_sum", ascending=False).head(MAX_POINTS_ON_MAP)
fig = px.scatter_map(
    plot_df,
    lat="lat",
    lon="lon",
    color="w_log",
    size="w_log",
    hover_name="grid_uid",
    hover_data={
        "tau_filled_min_sum": ":.1f",
        "tau_strict_min_sum": ":.1f",
        "tau_fill_min_sum": ":.1f",
        "fill_share": ":.2%",
        "w_log": False,
        "lat": False,
        "lon": False,
    },
    zoom=9,
    height=650,
)
fig.update_layout(map_style="carto-positron", margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig


,grid_uid,tau_filled_min_sum,tau_strict_min_sum,tau_fill_min_sum,n_user_rows,col,row,fill_share,w_log
2396,grid_440300_13_41,7624752.0,7624752.0,0.0,96362,13,41,0.000000,15.846910
801,grid_440300_49_23,6188829.0,6188829.0,0.0,96988,49,23,0.000000,15.638257
812,grid_440300_68_20,4901448.0,4901448.0,0.0,19157,68,20,0.000000,15.405041
3610,grid_440300_57_38,4449389.0,4449389.0,0.0,108147,57,38,0.000000,15.308278
3993,grid_440300_65_22,4396957.0,4396957.0,0.0,61119,65,22,0.000000,15.296423
808,grid_440300_67_18,4286594.0,4286594.0,0.0,83623,67,18,0.000000,15.271003
5960,grid_440300_77_24,3740261.0,3740261.0,0.0,38060,77,24,0.000000,15.134666
4772,grid_440300_77_22,3702751.0,3702751.0,0.0,73586,77,22,0.000000,15.124587
4367,grid_440300_77_23,3652080.0,3651840.0,240.0,26428,77,23,0.000066,15.110808
4767,grid_440300_64_22,3569064.0,3569064.0,0.0,40181,64,22,0.000000,15.087814


## User classification (regularity + activity space) + SES

This section computes per-user mobility metrics from the exported `OUT_FILE` and links them with SES fields from your `uuid.csv`.

- **Regularity**: lower grid diversity (entropy) ⇒ more regular.
- **Activity space**: radius of gyration (meters) based on visited grid centroids.

### What does `entropy` mean here?

For each user (and each meal `window` if you compute by window), we aggregate exposure weight per grid:

- `w_g = sum(tau_filled_min)` for grid `g`
- `p_g = w_g / sum_g w_g` (so `p_g` sums to 1)

Then we compute **Shannon entropy** (natural log, unit = *nats*):

- `entropy = -Σ_g p_g * ln(p_g)`

Interpretation:

- `entropy = 0` if almost all exposure is in one grid (very regular / concentrated)
- `entropy` increases as exposure spreads across many grids; the maximum is `ln(n_grids)` when exposure is uniform
- We also compute `entropy_norm = entropy / ln(n_grids)` (0–1) and `effective_grids = exp(entropy)` ("how many equally-weighted grids")
- In the code, `regularity_score = 1 - entropy_norm` (higher = more regular)


In [10]:
import os
import json
from pathlib import Path

import numpy as np
import pandas as pd
import duckdb
from pyproj import CRS, Transformer
import plotly.express as px
import time
from contextlib import contextmanager

if "step" not in globals():
    @contextmanager
    def step(name: str):
        t0 = time.perf_counter()
        print(f"[START] {name}")
        try:
            yield
        finally:
            dt = time.perf_counter() - t0
            print(f"[ END ] {name} (dt={dt:,.1f}s)")

# --- Config ---
IS_WEEKEND_METRICS = "all"  # True | False | "all"
COMPUTE_BY_WINDOW_METRICS = True  # set False to only compute window='all'
RECOMPUTE_USER_METRICS = False

USER_METRICS_ALL_FILE = OUT_CITY_DIR / f"user_metrics_{CITY}_all.parquet"
USER_METRICS_BY_WINDOW_FILE = OUT_CITY_DIR / f"user_metrics_{CITY}_by_window.parquet"

# SES table (uuid.csv) for linking.
_default_ses = (
    (Path(PROJECT_ROOT) / "data" / "jike" / "uuid.csv")
    if "PROJECT_ROOT" in globals()
    else Path(r"E:\\OneDrive\\HKU\\Project\\202512_EFE\\data\\jike\\uuid.csv")
)
SES_TABLE = Path(os.environ.get("POI_VISIT_SES_TABLE", str(_default_ses)))
SES_COLS_CANDIDATES = [
    "income_level",
    "income_price",
    "education_level",
    "consumption_level",
    "phone_price",
    "residence_p_price",
    "work_p_price",
    "live_p_price",
    # Home/work coordinates for mapping (naming varies by dataset version)
    "work_longitude",
    "work_latitude",
    "work_center_lnt",
    "work_center_lat",
    "live_longitude",
    "live_latitude",
    "live_center_lnt",
    "live_center_lat",
    "residence_longitude",
    "residence_latitude",
]


def _load_grid_transform(meta_path: Path) -> dict:
    obj = json.loads(Path(meta_path).read_text(encoding="utf-8"))
    data = obj.get("data", {})

    cell_size_m = float(obj.get("grid_size_m") or obj.get("cell_size_m") or 500.0)
    grid_crs = obj.get("crs") or obj.get("grid_crs") or (f"EPSG:{obj.get('epsg')}" if obj.get("epsg") else None)
    if grid_crs is None:
        raise ValueError(f"grid_meta missing `crs/epsg`: {meta_path}")

    ids = data.get("grid_uid") or data.get("grid_id")
    center_x = np.asarray(data.get("center_x", []), dtype=float)
    center_y = np.asarray(data.get("center_y", []), dtype=float)
    if ids is None or len(ids) == 0 or center_x.size == 0 or center_y.size == 0:
        raise ValueError(f"grid_meta missing `data.grid_uid/grid_id` or `data.center_x/center_y`: {meta_path}")

    ids = pd.Series(ids, dtype="string")
    parts = ids.str.extract(r".*_(?P<col>\d+)_(?P<row>\d+)$")
    col_m = parts["col"].astype(int).to_numpy()
    row_m = parts["row"].astype(int).to_numpy()

    origin_x = float(np.median(center_x - (col_m + 0.5) * cell_size_m))
    origin_y = float(np.median(center_y - (row_m + 0.5) * cell_size_m))

    transformer = Transformer.from_crs(
        CRS.from_user_input(grid_crs),
        CRS.from_user_input("EPSG:4326"),
        always_xy=True,
    )
    return {
        "cell_size_m": cell_size_m,
        "grid_crs": grid_crs,
        "origin_x": origin_x,
        "origin_y": origin_y,
        "transformer": transformer,
    }


def _user_metrics_sql(*, group_by_window: bool, origin_x: float, origin_y: float, cell_size_m: float, where_sql: str) -> str:
    win_select = '"window" as win' if group_by_window else "'all' as win"
    return f'''
    with base as (
      select
        uuid,
        {win_select},
        regexp_extract(grid_uid, '.*_(\\d+)_\\d+$', 1)::int as col,
        regexp_extract(grid_uid, '.*_\\d+_(\\d+)$', 1)::int as row,
        sum(tau_filled_min) as w
      from read_parquet(?)
      {where_sql}
      group by 1,2,3,4
    ),
    coords as (
      select
        *,
        {origin_x} + (col + 0.5) * {cell_size_m} as x,
        {origin_y} + (row + 0.5) * {cell_size_m} as y,
        w / sum(w) over(partition by uuid, win) as p
      from base
    ),
    reg as (
      select
        uuid,
        win,
        sum(w) as w_sum,
        count(*) as n_grids,
        max(w) / sum(w) as top1_share,
        -sum(p * ln(p)) as entropy
      from coords
      group by 1,2
    ),
    means as (
      select
        uuid,
        win,
        sum(w * x) / sum(w) as mean_x,
        sum(w * y) / sum(w) as mean_y,
        max(x) as max_x,
        min(x) as min_x,
        max(y) as max_y,
        min(y) as min_y
      from coords
      group by 1,2
    ),
    rg as (
      select
        c.uuid,
        c.win,
        sqrt(
          sum(c.w * ((c.x - m.mean_x) * (c.x - m.mean_x) + (c.y - m.mean_y) * (c.y - m.mean_y))) / r.w_sum
        ) as rg_m
      from coords c
      join means m using(uuid, win)
      join reg r using(uuid, win)
      group by 1,2, r.w_sum, m.mean_x, m.mean_y
    )
    select
      r.uuid,
      r.win as window,
      r.w_sum,
      r.n_grids,
      r.top1_share,
      r.entropy,
      case when r.n_grids > 1 then r.entropy / ln(r.n_grids) else 0 end as entropy_norm,
      exp(r.entropy) as effective_grids,
      sqrt(pow(m.max_x - m.min_x, 2) + pow(m.max_y - m.min_y, 2)) as bbox_diag_m,
      g.rg_m
    from reg r
    join means m using(uuid, win)
    join rg g using(uuid, win)
    '''


def _ensure_con() -> duckdb.DuckDBPyConnection:
    if "con" in globals():
        return con
    c = duckdb.connect()
    c.execute(f"PRAGMA threads={max(1, (os.cpu_count() or 8) - 1)}")
    return c


def _compute_user_metrics_parquet(
    *,
    con: duckdb.DuckDBPyConnection,
    out_file: Path,
    cache_path: Path,
    group_by_window: bool,
    grid_t: dict,
    is_weekend: bool | str,
    recompute: bool,
):
    cache_path.parent.mkdir(parents=True, exist_ok=True)
    if cache_path.exists() and not recompute:
        return

    where = []
    params: list[object] = [str(out_file)]
    if is_weekend != "all":
        where.append("is_weekend = ?")
        params.append(bool(is_weekend))
    where_sql = ("where " + " and ".join(where)) if where else ""

    sql = _user_metrics_sql(
        group_by_window=group_by_window,
        origin_x=float(grid_t["origin_x"]),
        origin_y=float(grid_t["origin_y"]),
        cell_size_m=float(grid_t["cell_size_m"]),
        where_sql=where_sql,
    )

    with step(f"compute_user_metrics → {cache_path.name}"):
        con.execute(
            f"COPY ({sql}) TO '{cache_path.as_posix()}' (FORMAT PARQUET)",
            params,
        )


def _qcut_named(series: pd.Series, q: int, names3: list[str], prefix: str) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    if s.nunique(dropna=True) < 2:
        return pd.Series(pd.NA, index=series.index)

    qcat = pd.qcut(s, q=q, duplicates="drop")
    n_bins = len(qcat.cat.categories)
    if n_bins == len(names3):
        return qcat.cat.rename_categories(names3)
    return qcat.cat.rename_categories([f"{prefix}{i + 1}" for i in range(n_bins)])


if GRID_META_VIZ is None:
    raise FileNotFoundError(
        "GRID_META_VIZ not found. Set env var POI_VISIT_GRID_META_VIZ to your `grid_meta_<city>.json`."
    )

GRID_T = _load_grid_transform(Path(GRID_META_VIZ))
con = _ensure_con()

_compute_user_metrics_parquet(
    con=con,
    out_file=OUT_FILE,
    cache_path=USER_METRICS_ALL_FILE,
    group_by_window=False,
    grid_t=GRID_T,
    is_weekend=IS_WEEKEND_METRICS,
    recompute=RECOMPUTE_USER_METRICS,
)
if COMPUTE_BY_WINDOW_METRICS:
    _compute_user_metrics_parquet(
        con=con,
        out_file=OUT_FILE,
        cache_path=USER_METRICS_BY_WINDOW_FILE,
        group_by_window=True,
        grid_t=GRID_T,
        is_weekend=IS_WEEKEND_METRICS,
        recompute=RECOMPUTE_USER_METRICS,
    )

user_metrics_all = con.execute("select * from read_parquet(?)", [str(USER_METRICS_ALL_FILE)]).fetchdf()
user_metrics_all = user_metrics_all.sort_values("w_sum", ascending=False).reset_index(drop=True)

# Regularity (high = more concentrated/regular)
user_metrics_all["regularity_score"] = 1.0 - user_metrics_all["entropy_norm"]
user_metrics_all["regularity_group"] = _qcut_named(
    user_metrics_all["regularity_score"], q=3, names3=["low", "mid", "high"], prefix="reg_q"
)
user_metrics_all["activity_space_group"] = _qcut_named(
    user_metrics_all["rg_m"], q=3, names3=["small", "medium", "large"], prefix="space_q"
)
user_metrics_all["user_group"] = (
    user_metrics_all["regularity_group"].astype(str)
    + " / "
    + user_metrics_all["activity_space_group"].astype(str)
)

display(user_metrics_all.head(10))
display(
    user_metrics_all[["w_sum", "n_grids", "top1_share", "entropy_norm", "rg_m", "bbox_diag_m"]].describe(
        percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
    )
)
display(user_metrics_all["user_group"].value_counts())

# --- SES merge (optional) ---
user_metrics_all_ses = user_metrics_all
if SES_TABLE.exists():
    with step(f"load_ses → {SES_TABLE.name}"):
        if SES_TABLE.suffix.lower() == ".parquet":
            ses = pd.read_parquet(SES_TABLE)
        else:
            header = pd.read_csv(SES_TABLE, nrows=0)
            usecols = ["uuid"] + [c for c in SES_COLS_CANDIDATES if c in header.columns]
            ses = pd.read_csv(SES_TABLE, usecols=usecols, on_bad_lines="skip")

    user_metrics_all_ses = user_metrics_all.merge(ses, on="uuid", how="left")

    num_cols = [
        c
        for c in ["income_price", "phone_price", "residence_p_price", "work_p_price", "live_p_price"]
        if c in user_metrics_all_ses.columns
    ]
    if num_cols:
        display(
            user_metrics_all_ses.groupby("user_group")[num_cols]
            .mean(numeric_only=True)
            .sort_index()
            .round(2)
        )
    if "income_level" in user_metrics_all_ses.columns:
        display(pd.crosstab(user_metrics_all_ses["user_group"], user_metrics_all_ses["income_level"], normalize="index"))
else:
    print(f"SES_TABLE not found: {SES_TABLE} (set env var POI_VISIT_SES_TABLE if needed)")

# --- Correlation: income_price vs regularity ---
if "income_price" in user_metrics_all_ses.columns:
    df_corr = user_metrics_all_ses[["income_price", "regularity_score", "entropy_norm", "regularity_group"]].copy()
    df_corr["income_price"] = pd.to_numeric(df_corr["income_price"], errors="coerce")
    df_corr = df_corr.dropna(subset=["income_price", "regularity_score", "entropy_norm"])

    n = len(df_corr)
    pearson_reg = df_corr["income_price"].corr(df_corr["regularity_score"], method="pearson")
    spearman_reg = df_corr["income_price"].corr(df_corr["regularity_score"], method="spearman")
    pearson_ent = df_corr["income_price"].corr(df_corr["entropy_norm"], method="pearson")
    spearman_ent = df_corr["income_price"].corr(df_corr["entropy_norm"], method="spearman")

    print(
        f"income_price vs regularity_score (n={n:,}): pearson={pearson_reg:.4f}, spearman={spearman_reg:.4f}"
    )
    print(f"income_price vs entropy_norm:            pearson={pearson_ent:.4f}, spearman={spearman_ent:.4f}")

    display(df_corr.groupby("regularity_group")["income_price"].describe())

    sample = df_corr.sample(min(80_000, n), random_state=0)
    fig = px.scatter(
        sample,
        x="regularity_score",
        y="income_price",
        color="regularity_group",
        opacity=0.25,
        labels={"regularity_score": "Regularity (1 - normalized entropy)"},
    )
    fig.update_layout(title="Income vs regularity (sample)")
    fig

# Optional quick plot (sample to keep it fast)
if "income_price" in user_metrics_all_ses.columns:
    sample = (
        user_metrics_all_ses[["rg_m", "income_price", "regularity_group"]]
        .dropna()
        .sample(min(50_000, len(user_metrics_all_ses)), random_state=0)
    )
    fig = px.scatter(sample, x="rg_m", y="income_price", color="regularity_group", opacity=0.35)
    fig.update_layout(title="Income vs activity space (sample)")
    fig


,uuid,window,w_sum,n_grids,top1_share,entropy,entropy_norm,effective_grids,bbox_diag_m,rg_m,regularity_score,regularity_group,activity_space_group,user_group
0,1bf1d10e-88d1-3682-aa8b-d2333ae23f72,all,13002196.0,457,0.057043,5.283460,0.862650,197.050583,84528.101836,14070.420508,0.137350,low,large,low / large
1,2d100368-95c8-3048-8eba-f9fc46f8ff59,all,11751.0,23,0.724874,0.920741,0.293651,2.511150,46327.637540,2696.284197,0.706349,high,medium,high / medium
2,0883cd2d-9392-3a48-9c65-3d7bcf1aa3bb,all,11717.0,7,0.622600,0.811196,0.416872,2.250598,31827.660926,1998.387842,0.583128,mid,medium,mid / medium
3,c59a7ba7-75e2-4012-be08-2552ddb4eb1c,all,10412.0,15,0.884268,0.563880,0.208223,1.757477,33541.019662,2237.041221,0.791777,high,medium,high / medium
4,00c8ae3d7bc6943cbf75c5506c0f1cfe,all,9792.0,8,0.965074,0.188822,0.090804,1.207826,20934.421415,868.833510,0.909196,high,small,high / small
5,32ce22b3-b033-352f-9cbd-1ad0422459fd,all,9735.0,10,0.924294,0.377590,0.163985,1.458765,15132.745950,2801.827431,0.836015,high,medium,high / medium
6,ecf67770-8370-30ed-b2ee-cd3a143b115d,all,9465.0,18,0.602747,1.165268,0.403155,3.206783,25144.581921,3706.711868,0.596845,mid,medium,mid / medium
7,a18241cf-a62d-43b4-9ee8-bba4b0d86563,all,9464.0,8,0.945266,0.262846,0.126402,1.300626,16007.810594,2707.152664,0.873598,high,medium,high / medium
8,f3afad5adf8d2e7be20e3825438725c6,all,9311.0,7,0.960692,0.194644,0.100027,1.214879,24413.111231,1333.038893,0.899973,high,small,high / small
9,1428142c-7937-31df-9ed5-476a14e3c2f7,all,8964.0,8,0.532129,0.868506,0.417663,2.383347,23505.318547,2301.223439,0.582337,mid,medium,mid / medium


,w_sum,n_grids,top1_share,entropy_norm,rg_m,bbox_diag_m
count,6.969410e+05,696941.000000,696941.000000,696941.000000,696941.000000,696941.000000
mean,1.300490e+03,8.651774,0.669284,0.493881,4817.410664,20283.877964
std,1.563118e+04,9.013640,0.242916,0.307028,4890.351846,18161.220103
min,1.000000e+00,1.000000,0.028609,0.000000,0.000000,0.000000
10%,2.100000e+01,1.000000,0.348025,0.000000,0.000000,0.000000
25%,9.000000e+01,2.000000,0.474506,0.271592,537.347330,2236.067977
50%,6.670000e+02,6.000000,0.655738,0.545325,3510.208877,17670.597047
75%,2.404000e+03,12.000000,0.903704,0.732385,7573.535886,33301.651611
90%,3.312000e+03,20.000000,1.000000,0.875944,11915.639326,46157.339611
max,1.300220e+07,457.000000,1.000000,1.000000,38018.116243,86609.756956


user_group
high / small     164132
low / large      127617
mid / medium     108521
mid / large       90169
low / medium      70138
high / medium     53654
low / small       34559
mid / small       33623
high / large      14528
Name: count, dtype: int64

[START] load_ses → uuid.csv (RAM=445 MB)


C:\Users\weipd\AppData\Local\Temp\ipykernel_29184\2496607508.py:292: DtypeWarning:

Columns (137) have mixed types. Specify dtype option on import or set low_memory=False.



...

,income_price,phone_price,residence_p_price,work_p_price,live_p_price
user_group,,,,,
high / large,7809.82,4556.68,38396.90,42907.01,40743.90
high / medium,7739.80,4693.76,43361.21,49474.44,46644.36
high / small,6898.93,4912.05,30079.32,35196.13,32281.45
low / large,8181.24,5168.91,33651.45,33399.48,36891.55
low / medium,7769.19,5443.70,38997.89,38946.03,42944.88
low / small,6996.17,4885.25,31682.31,36068.90,34550.28
mid / large,8334.47,5232.93,41508.96,42948.66,46073.33
mid / medium,8194.70,5409.04,47501.20,51732.14,53002.08
mid / small,7315.02,4736.60,44165.68,49452.07,48093.67


income_level,1,2,3,4,5
user_group,,,,,
high / large,0.032558,0.068420,0.149367,0.261908,0.487748
high / medium,0.031498,0.068420,0.158460,0.260596,0.481027
high / small,0.037762,0.081307,0.155015,0.240495,0.485420
low / large,0.022011,0.044876,0.125328,0.224453,0.583331
low / medium,0.023625,0.048889,0.123941,0.238786,0.564758
low / small,0.032958,0.070488,0.149860,0.252814,0.493880
mid / large,0.018931,0.039382,0.118999,0.243321,0.579368
mid / medium,0.017729,0.036666,0.122695,0.245473,0.577437
mid / small,0.029206,0.064628,0.154953,0.270708,0.480504


income_price vs regularity_score (n=696,941): pearson=-0.0680, spearman=-0.0651
income_price vs entropy_norm:            pearson=0.0680, spearman=0.0651


C:\Users\weipd\AppData\Local\Temp\ipykernel_29184\2496607508.py:330: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,count,mean,std,min,25%,50%,75%,max
regularity_group,,,,,,,,
low,232314.0,7880.549248,5374.289793,5.0,4500.0,6688.0,9891.0,834956.0
mid,232313.0,8121.631953,4977.075988,5.0,5005.0,7049.0,10075.0,71737.0
high,232314.0,7150.092694,4672.923412,1875.0,3929.0,6089.0,8965.0,63584.0


## Inspect a specific user (uuid / iloc)

Map shows `grid_uid` exposure weights by meal window (animation frames).

In [11]:
from pathlib import Path

import numpy as np
import pandas as pd
import duckdb
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Ensure Plotly renders inside VSCode/Colab notebooks.
if "IN_COLAB" in globals() and IN_COLAB and "colab" in pio.renderers:
    pio.renderers.default = "colab"
elif "vscode" in pio.renderers:
    pio.renderers.default = "vscode"
elif "plotly_mimetype" in pio.renderers:
    pio.renderers.default = "plotly_mimetype"

# Pick a user either by uuid or by iloc in `user_metrics_all_ses` (sorted by w_sum desc).
INSPECT_UUID: str | None = None
INSPECT_ILOC: int | None = 2
INSPECT_IS_WEEKEND = "all"  # True | False | "all"
MAX_USER_GRIDS_PER_WINDOW = 800
INSPECT_CELL_VERSION = "2026-01-11-scattermap-marker-fix-v3"
print("inspect_user cell version:", INSPECT_CELL_VERSION)


def _add_lonlat(df: pd.DataFrame, grid_uid_col: str = "grid_uid") -> pd.DataFrame:
    parts = df[grid_uid_col].astype("string").str.extract(r".*_(?P<col>\d+)_(?P<row>\d+)$")
    df = df.copy()
    df["col"] = parts["col"].astype(int)
    df["row"] = parts["row"].astype(int)

    cell_size_m = float(GRID_T["cell_size_m"])
    origin_x = float(GRID_T["origin_x"])
    origin_y = float(GRID_T["origin_y"])

    df["x"] = origin_x + (df["col"].to_numpy() + 0.5) * cell_size_m
    df["y"] = origin_y + (df["row"].to_numpy() + 0.5) * cell_size_m
    lon, lat = GRID_T["transformer"].transform(df["x"].to_numpy(), df["y"].to_numpy())
    df["lon"] = lon
    df["lat"] = lat
    return df


def _fetch_user_grid(con: duckdb.DuckDBPyConnection, out_file: Path, uuid: str, is_weekend: bool | str = "all") -> pd.DataFrame:
    where = ["uuid = ?"]
    params: list[object] = [str(out_file), uuid]
    if is_weekend != "all":
        where.append("is_weekend = ?")
        params.append(bool(is_weekend))
    where_sql = " and ".join(where)

    return con.execute(
        f'''
        select
          grid_uid,
          "window" as window,
          sum(tau_filled_min) as w
        from read_parquet(?)
        where {where_sql}
        group by 1,2
        ''',
        params,
    ).fetchdf()


def _first_float(row: pd.Series, cols: list[str]) -> float | None:
    for c in cols:
        if c not in row.index:
            continue
        v = row[c]
        if pd.isna(v):
            continue
        try:
            return float(v)
        except Exception:
            continue
    return None


def _valid_lonlat(lon: float, lat: float) -> bool:
    return (-180.0 <= lon <= 180.0) and (-90.0 <= lat <= 90.0)


def _user_places(uuid: str) -> pd.DataFrame:
    rows = user_metrics_all_ses.loc[user_metrics_all_ses["uuid"] == uuid]
    if rows.empty:
        return pd.DataFrame(columns=["place", "lon", "lat"])
    row = rows.iloc[0]

    pts: list[dict] = []

    # Prefer `live_*` as home; fall back to `residence_*` if missing.
    home_lon = _first_float(row, ["live_center_lnt", "live_longitude", "residence_longitude"])
    home_lat = _first_float(row, ["live_center_lat", "live_latitude", "residence_latitude"])
    if home_lon is not None and home_lat is not None and _valid_lonlat(home_lon, home_lat):
        pts.append({"place": "home", "lon": home_lon, "lat": home_lat})

    work_lon = _first_float(row, ["work_center_lnt", "work_longitude"])
    work_lat = _first_float(row, ["work_center_lat", "work_latitude"])
    if work_lon is not None and work_lat is not None and _valid_lonlat(work_lon, work_lat):
        pts.append({"place": "work", "lon": work_lon, "lat": work_lat})

    return pd.DataFrame(pts)


def inspect_user(
    *,
    uuid: str | None = None,
    iloc: int | None = None,
    is_weekend: bool | str = "all",
    max_grids_per_window: int = 800,
) -> tuple[str, pd.DataFrame, px.Figure]:
    if uuid is None:
        if iloc is None:
            raise ValueError("Provide either uuid=... or iloc=...")
        uuid = str(user_metrics_all_ses.iloc[int(iloc)]["uuid"])

    display(user_metrics_all_ses.loc[user_metrics_all_ses["uuid"] == uuid].T)

    ug = _fetch_user_grid(con, OUT_FILE, uuid, is_weekend=is_weekend)
    if ug.empty:
        raise ValueError(f"No rows for uuid={uuid!r} (check is_weekend filter and OUT_FILE)")

    all_ug = ug.groupby("grid_uid", as_index=False)["w"].sum()
    all_ug["window"] = "all"
    plot_df = pd.concat([ug, all_ug], ignore_index=True)

    plot_df["w_share"] = plot_df["w"] / plot_df.groupby("window")["w"].transform("sum")
    plot_df["w_log"] = np.log1p(plot_df["w"])

    plot_df_full = _add_lonlat(plot_df)
    plot_df_map = plot_df_full.sort_values(["window", "w"], ascending=[True, False])
    plot_df_map = plot_df_map.groupby("window", as_index=False).head(max_grids_per_window)

    places = _user_places(uuid)

    # Weighted center for a reasonable initial view.
    center_lon = float(np.average(plot_df_full["lon"], weights=plot_df_full["w"]))
    center_lat = float(np.average(plot_df_full["lat"], weights=plot_df_full["w"]))
    if not places.empty:
        center_lon = float(np.mean([center_lon, places["lon"].mean()]))
        center_lat = float(np.mean([center_lat, places["lat"].mean()]))

    fig = px.scatter_map(
        plot_df_map,
        lat="lat",
        lon="lon",
        color="w_share",
        size="w_log",
        animation_frame="window",
        hover_name="grid_uid",
        hover_data={
            "w": ":.1f",
            "w_share": ":.2%",
            "w_log": False,
            "lat": False,
            "lon": False,
            "col": True,
            "row": True,
        },
        center={"lat": center_lat, "lon": center_lon},
        zoom=10,
        height=650,
    )

    if not places.empty:
        display(places)

        styles = {
            # Use simple circles + labels (H/W) for reliable rendering.
            "home": {"color": "#2ca02c", "text": "H"},
            "work": {"color": "#d62728", "text": "W"},
        }

        place_traces: list[go.Scattermap] = []
        for place, g in places.groupby("place", sort=True):
            st = styles.get(str(place), {"color": "#1f77b4", "text": ""})
            tr = go.Scattermap(
                lat=g["lat"],
                lon=g["lon"],
                mode="markers+text",
                text=[st.get("text", "")] * len(g),
                textposition="top center",
                textfont={"size": 12, "color": "black"},
                subplot="map",
                skip_invalid=True,
                marker={
                    "size": 13,
                    "color": st["color"],
                    "symbol": "circle",
                    "allowoverlap": True,
                },
                name=str(place),
                hovertemplate=(
                    f"{place}<br>lat=%{{lat:.6f}}<br>lon=%{{lon:.6f}}<extra></extra>"
                ),
            )
            fig.add_trace(tr)
            place_traces.append(tr)

        # Make sure the place markers stay visible and on top for every animation frame.
        if fig.frames and place_traces:
            for fr in fig.frames:
                fr.data = tuple(fr.data) + tuple(place_traces)

        # Redraw on frame switch so z-order is preserved (important when points overlap).
        if fig.layout.updatemenus:
            for um in fig.layout.updatemenus:
                if um.buttons is None:
                    continue
                for b in um.buttons:
                    if not isinstance(b.args, (list, tuple)) or len(b.args) < 2 or not isinstance(b.args[1], dict):
                        continue
                    frame = b.args[1].get("frame")
                    if isinstance(frame, dict):
                        frame["redraw"] = True
        if fig.layout.sliders:
            for sl in fig.layout.sliders:
                if sl.steps is None:
                    continue
                for st in sl.steps:
                    if not isinstance(st.args, (list, tuple)) or len(st.args) < 2 or not isinstance(st.args[1], dict):
                        continue
                    frame = st.args[1].get("frame")
                    if isinstance(frame, dict):
                        frame["redraw"] = True
    else:
        print(
            "No valid home/work coords for this user (check live_center_lnt/lat & work_center_lnt/lat in SES_TABLE)."
        )
    fig.update_layout(map_style="carto-positron", margin={"r": 0, "t": 30, "l": 0, "b": 0})
    fig.update_layout(title=f"uuid={uuid} | is_weekend={is_weekend}")
    return uuid, plot_df_full, fig


uuid_i, plot_df_i, fig_i = inspect_user(
    uuid=INSPECT_UUID,
    iloc=INSPECT_ILOC,
    is_weekend=INSPECT_IS_WEEKEND,
    max_grids_per_window=MAX_USER_GRIDS_PER_WINDOW,
)
display(plot_df_i.sort_values(["window", "w"], ascending=[True, False]).head(30))
fig_i


inspect_user cell version: 2026-01-11-scattermap-marker-fix-v3


,2
uuid,0883cd2d-9392-3a48-9c65-3d7bcf1aa3bb
window,all
w_sum,11717.0
n_grids,7
top1_share,0.6226
entropy,0.811196
entropy_norm,0.416872
effective_grids,2.250598
bbox_diag_m,31827.660926
rg_m,1998.387842


,place,lon,lat
0,home,114.022547,22.527186
1,work,113.984765,22.527912


,grid_uid,window,w,w_share,w_log,col,row,x,y,lon,lat
14,grid_440300_50_19,all,7295.0,0.622600,8.895082,50,19,3.642683e+07,2.520295e+06,113.982975,22.528295
13,grid_440300_49_23,all,3965.0,0.338397,8.285513,49,23,3.642633e+07,2.522295e+06,113.979337,22.546404
15,grid_440300_58_19,all,390.0,0.033285,5.968708,58,19,3.643083e+07,2.520295e+06,114.021375,22.526130
10,grid_440300_14_63,all,35.0,0.002987,3.583519,14,63,3.640883e+07,2.542295e+06,113.822741,22.734236
16,grid_440300_58_22,all,15.0,0.001280,2.772589,58,22,3.643083e+07,2.521795e+06,114.022251,22.539507
11,grid_440300_27_24,all,12.0,0.001024,2.564949,27,24,3.641533e+07,2.522795e+06,113.873985,22.556776
12,grid_440300_40_17,all,5.0,0.000427,1.791759,40,17,3.642183e+07,2.519295e+06,113.934389,22.522069
5,grid_440300_50_19,dinner,4220.0,0.592946,8.347827,50,19,3.642683e+07,2.520295e+06,113.982975,22.528295
1,grid_440300_49_23,dinner,2622.0,0.368414,7.872074,49,23,3.642633e+07,2.522295e+06,113.979337,22.546404
0,grid_440300_58_19,dinner,240.0,0.033722,5.484797,58,19,3.643083e+07,2.520295e+06,114.021375,22.526130


## 全样本：用餐地点到家/工作距离分布

下面两张图统计 **全体用户** 的用餐网格地点（按 `uuid × grid × window` 聚合）到 **居住地(home)** 和 **工作地(work)** 的距离分布。

- 横轴：距离（km）
- 纵轴：地点频率占比（或按暴露时长占比）
- 图形：柱状图 + 平滑拟合线


In [12]:
# =========================================================
# Font setup: Optima LT Pro (matplotlib)
# =========================================================
import matplotlib as mpl
import matplotlib.font_manager as fm

# 1️⃣ 如果系统已安装 Optima LT Pro（macOS / 部分 Windows）
mpl.rcParams["font.family"] = "Optima LT Pro"

# 2️⃣ 保险起见：明确指定字体文件（推荐）
# 把路径换成你机器上的真实路径
font_path = "E:\\OneDrive\\HKU\Project\\202512_EFE\\OptimaLTPro-Roman.otf"   # or .ttf
fm.fontManager.addfont(font_path)
    
optima = fm.FontProperties(fname=font_path)
mpl.rcParams["font.family"] = optima.get_name()

# 3️⃣ 数学/符号也尽量统一风格
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = optima.get_name()
mpl.rcParams["mathtext.it"] = optima.get_name()
mpl.rcParams["mathtext.bf"] = optima.get_name()


<>:12: SyntaxWarning:

"\P" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\P"? A raw string is also an option.

<>:12: SyntaxWarning:

"\P" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\P"? A raw string is also an option.

C:\Users\weipd\AppData\Local\Temp\ipykernel_29184\386311867.py:12: SyntaxWarning:

"\P" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\P"? A raw string is also an option.



In [13]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import duckdb
import plotly.graph_objects as go

# --- Config ---
DIST_POP_WINDOW = "all"  # "lunch" | "dinner" | "all"
DIST_POP_IS_WEEKEND = "all"  # True | False | "all"
DIST_POP_WEIGHT_MODE = "location_share"  # "location_share" | "weight_share"
DIST_POP_BIN_KM = 1.0
DIST_POP_MAX_KM = 40.0
DIST_POP_SMOOTH_SIGMA_BINS = 1.5
RECOMPUTE_DIST_POP = False

assert OUT_FILE.exists(), f"Missing output parquet: {OUT_FILE}"
assert SES_TABLE.exists(), f"Missing SES_TABLE: {SES_TABLE}"
if GRID_META_VIZ is None:
    raise FileNotFoundError(
        "GRID_META_VIZ not found. Set env var POI_VISIT_GRID_META_VIZ to your `grid_meta_<city>.json`."
    )

DIST_POP_CACHE = OUT_CITY_DIR / f"pop_meal_dist_{CITY}_{DIST_POP_WINDOW}_{DIST_POP_IS_WEEKEND}.parquet"


def _gaussian_smooth(y: np.ndarray, sigma_bins: float) -> np.ndarray:
    if sigma_bins <= 0:
        return y
    radius = int(np.ceil(3 * sigma_bins))
    x = np.arange(-radius, radius + 1)
    k = np.exp(-0.5 * (x / sigma_bins) ** 2)
    k = k / k.sum()
    return np.convolve(y, k, mode="same")


def _grid_lookup_from_meta(meta_path: Path) -> pd.DataFrame:
    obj = json.loads(Path(meta_path).read_text(encoding="utf-8"))
    data = obj.get("data", {})

    ids = pd.Series(data.get("grid_uid") or data.get("grid_id"), dtype="string")
    parts = ids.str.extract(r".*_(?P<col>\d+)_(?P<row>\d+)$")

    lon = np.asarray(data.get("center_lon", []), dtype=float)
    lat = np.asarray(data.get("center_lat", []), dtype=float)
    if lon.size == 0 or lat.size == 0:
        raise ValueError(f"grid_meta missing `data.center_lon/center_lat`: {meta_path}")

    df = pd.DataFrame(
        {
            "col": parts["col"].astype(int),
            "row": parts["row"].astype(int),
            "lon": lon,
            "lat": lat,
        }
    )
    df = df.dropna(subset=["col", "row", "lon", "lat"]).drop_duplicates(["col", "row"])
    return df


def _build_ses_ll() -> pd.DataFrame:
    # Prefer the already-merged per-user table (smaller than the raw SES csv)
    if "user_metrics_all_ses" in globals():
        src = user_metrics_all_ses
    elif "ses" in globals():
        src = ses
    else:
        header = pd.read_csv(SES_TABLE, nrows=0)
        usecols = [
            "uuid",
            "live_center_lnt",
            "live_center_lat",
            "work_center_lnt",
            "work_center_lat",
            "live_longitude",
            "live_latitude",
            "work_longitude",
            "work_latitude",
            "residence_longitude",
            "residence_latitude",
        ]
        usecols = [c for c in usecols if c in header.columns]
        src = pd.read_csv(SES_TABLE, usecols=usecols, on_bad_lines="skip")

    def pick(cols: list[str]) -> str | None:
        for c in cols:
            if c in src.columns:
                return c
        return None

    home_lon_col = pick(["live_center_lnt", "live_longitude", "residence_longitude"])
    home_lat_col = pick(["live_center_lat", "live_latitude", "residence_latitude"])
    work_lon_col = pick(["work_center_lnt", "work_longitude"])
    work_lat_col = pick(["work_center_lat", "work_latitude"])

    if home_lon_col is None or home_lat_col is None:
        raise ValueError("Missing home coords in SES data (need live_center_lnt/lat or live_longitude/lat)")
    if work_lon_col is None or work_lat_col is None:
        raise ValueError("Missing work coords in SES data (need work_center_lnt/lat or work_longitude/lat)")

    df = pd.DataFrame(
        {
            "uuid": src["uuid"].astype("string"),
            "home_lon": pd.to_numeric(src[home_lon_col], errors="coerce"),
            "home_lat": pd.to_numeric(src[home_lat_col], errors="coerce"),
            "work_lon": pd.to_numeric(src[work_lon_col], errors="coerce"),
            "work_lat": pd.to_numeric(src[work_lat_col], errors="coerce"),
        }
    )

    # Keep rows that have at least one valid point.
    home_ok = df["home_lon"].between(-180, 180) & df["home_lat"].between(-90, 90)
    work_ok = df["work_lon"].between(-180, 180) & df["work_lat"].between(-90, 90)
    df.loc[~home_ok, ["home_lon", "home_lat"]] = np.nan
    df.loc[~work_ok, ["work_lon", "work_lat"]] = np.nan
    df = df.dropna(subset=["home_lon", "home_lat", "work_lon", "work_lat"], how="all")
    return df


def _ensure_con_local() -> duckdb.DuckDBPyConnection:
    if "_ensure_con" in globals():
        return _ensure_con()
    c = duckdb.connect()
    c.execute(f"PRAGMA threads={max(1, (os.cpu_count() or 8) - 1)}")
    return c


con = _ensure_con_local()
grid_ll = _grid_lookup_from_meta(Path(GRID_META_VIZ))
ses_ll = _build_ses_ll()

con.register("grid_ll", grid_ll)
con.register("ses_ll", ses_ll)

if DIST_POP_CACHE.exists() and not RECOMPUTE_DIST_POP:
    bins_df = con.execute("select * from read_parquet(?)", [str(DIST_POP_CACHE)]).fetchdf()
else:
    where = []
    params: list[object] = [str(OUT_FILE)]
    if DIST_POP_WINDOW != "all":
        where.append('"window" = ?')
        params.append(DIST_POP_WINDOW)
    if DIST_POP_IS_WEEKEND != "all":
        where.append("is_weekend = ?")
        params.append(bool(DIST_POP_IS_WEEKEND))
    where_sql = ("where " + " and ".join(where)) if where else ""

    bin_km = float(DIST_POP_BIN_KM)
    max_km = float(DIST_POP_MAX_KM)

    sql = f'''
    with base as (
      select
        uuid,
        "window" as win,
        regexp_extract(grid_uid, '.*_(\\d+)_\\d+$', 1)::int as col,
        regexp_extract(grid_uid, '.*_\\d+_(\\d+)$', 1)::int as row,
        sum(tau_filled_min) as w_sum
      from read_parquet(?)
      {where_sql}
      group by 1,2,3,4
    ),
    joined as (
      select
        b.w_sum,
        g.lon,
        g.lat,
        s.home_lon,
        s.home_lat,
        s.work_lon,
        s.work_lat
      from base b
      join grid_ll g using(col, row)
      join ses_ll s using(uuid)
    ),
    dist as (
      select
        'home' as place,
        6371.0 * 2 * asin(least(1.0, sqrt(
          pow(sin(radians(home_lat - lat) / 2), 2)
          + cos(radians(lat)) * cos(radians(home_lat))
          * pow(sin(radians(home_lon - lon) / 2), 2)
        ))) as dist_km,
        1::bigint as n_locs,
        w_sum as w_sum
      from joined
      where home_lon is not null and home_lat is not null

      union all

      select
        'work' as place,
        6371.0 * 2 * asin(least(1.0, sqrt(
          pow(sin(radians(work_lat - lat) / 2), 2)
          + cos(radians(lat)) * cos(radians(work_lat))
          * pow(sin(radians(work_lon - lon) / 2), 2)
        ))) as dist_km,
        1::bigint as n_locs,
        w_sum as w_sum
      from joined
      where work_lon is not null and work_lat is not null
    ),
    binned as (
      select
        place,
        floor(dist_km / {bin_km}) * {bin_km} as bin_km,
        sum(n_locs) as n_locs,
        sum(w_sum) as w_sum
      from dist
      where dist_km between 0 and {max_km}
      group by 1,2
    )
    select * from binned
    order by place, bin_km
    '''

    # Cache the binned counts so reruns are instant.
    con.execute(f"COPY ({sql}) TO '{DIST_POP_CACHE.as_posix()}' (FORMAT PARQUET)", params)
    bins_df = con.execute("select * from read_parquet(?)", [str(DIST_POP_CACHE)]).fetchdf()

bins_df["bin_km"] = pd.to_numeric(bins_df["bin_km"], errors="coerce")
bins_df["n_locs"] = pd.to_numeric(bins_df["n_locs"], errors="coerce")
bins_df["w_sum"] = pd.to_numeric(bins_df["w_sum"], errors="coerce")

y_title = "Share of meal locations (%)" if DIST_POP_WEIGHT_MODE == "location_share" else "Share of meal exposure (%)"


def _plot_place(place: str) -> go.Figure | None:
    dfp = bins_df.loc[bins_df["place"] == place].copy()
    if dfp.empty:
        print(f"No rows for place={place!r}")
        return None

    bin_km = float(DIST_POP_BIN_KM)
    max_km = float(DIST_POP_MAX_KM)
    bin_starts = np.arange(0.0, max_km, bin_km)
    centers = bin_starts + bin_km / 2

    val_col = "n_locs" if DIST_POP_WEIGHT_MODE == "location_share" else "w_sum"
    s = dfp.set_index("bin_km")[val_col]
    s = s.reindex(bin_starts, fill_value=0.0)
    share = s.to_numpy(dtype=float)
    if share.sum() > 0:
        share = share / share.sum()
    pct = 100.0 * share
    smooth = _gaussian_smooth(pct, sigma_bins=float(DIST_POP_SMOOTH_SIGMA_BINS))

    fig = go.Figure()
    fig.add_trace(go.Bar(x=centers, y=pct, name="bin %", opacity=0.65))
    fig.add_trace(go.Scatter(x=centers, y=smooth, mode="lines", name="smooth", line={"color": "black"}))
    fig.update_layout(
        title=f"Population distance to {place} | window={DIST_POP_WINDOW} | is_weekend={DIST_POP_IS_WEEKEND}",
        xaxis_title=f"Distance to {place} (km)",
        yaxis_title=y_title,
        height=380,
        margin={"r": 10, "t": 50, "l": 10, "b": 10},
        legend={"orientation": "h", "y": -0.2},
    )
    return fig


display(_plot_place("home"))
display(_plot_place("work"))


In [20]:
# =========================================================
# Weekday lunch/dinner: meal-location distance to Work/Home
# (2 figures: lunch + dinner)
# =========================================================
from pathlib import Path

import numpy as np
import pandas as pd
import duckdb
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Config ---
WD_BIN_KM = 0.5
WD_MAX_KM = 15.0
WD_WEIGHT_MODE = "weight_share"  # "weight_share" | "location_share"
WD_SMOOTH_SIGMA_BINS = 1.0  # in bins (smaller -> closer to bars)
WD_RECOMPUTE = False

WD_SAVE_PNG = True
WD_PNG_SCALE = 2
WD_PNG_DIR = OUT_CITY_DIR / "figures_meal_distance"
WD_PNG_DIR.mkdir(parents=True, exist_ok=True)

# Font (used for notebook display + PNG export; for PNG it must be available to Kaleido/Chrome).
WD_FONT_PATH = Path(r"E:\\OneDrive\\HKU\\Project\\202512_EFE\\OptimaLTPro-Roman.otf")
WD_FONT_FAMILY: str | None = None
if WD_FONT_PATH.exists():
    try:
        from matplotlib.font_manager import FontProperties

        WD_FONT_FAMILY = FontProperties(fname=str(WD_FONT_PATH)).get_name()
    except Exception:
        WD_FONT_FAMILY = None
if WD_FONT_FAMILY is None:
    WD_FONT_FAMILY = "Arial"
print("[font]", WD_FONT_FAMILY, "| file exists:", WD_FONT_PATH.exists())

assert OUT_FILE.exists(), f"Missing output parquet: {OUT_FILE}"


def _fmt_float(x: float) -> str:
    return str(x).replace(".", "p")


def _save_png(fig: go.Figure, stem: str) -> Path | None:
    if not WD_SAVE_PNG:
        return None
    out = WD_PNG_DIR / (
        f"{stem}_{WD_WEIGHT_MODE}_bin{_fmt_float(WD_BIN_KM)}km_max{_fmt_float(WD_MAX_KM)}km.png"
    )
    fig.write_image(str(out), scale=int(WD_PNG_SCALE))
    print("[OK] Saved:", out)
    return out


def _gaussian_smooth_edge(y: np.ndarray, sigma_bins: float) -> np.ndarray:
    """Gaussian smoothing with edge padding (avoids the start being biased low)."""
    if sigma_bins <= 0:
        return y
    radius = int(np.ceil(3 * sigma_bins))
    x = np.arange(-radius, radius + 1)
    k = np.exp(-0.5 * (x / sigma_bins) ** 2)
    k = k / k.sum()
    ypad = np.pad(y, (radius, radius), mode="edge")
    return np.convolve(ypad, k, mode="valid")


def _ensure_pop_bins(window: str, is_weekend: bool) -> pd.DataFrame:
    """Return binned distance distribution for (window, is_weekend). Cached to parquet."""
    global con, grid_ll, ses_ll
    if "con" not in globals():
        con = duckdb.connect()
    if "grid_ll" not in globals():
        if "_grid_lookup_from_meta" not in globals():
            raise RuntimeError("Please run the previous population-distance cell first (it defines grid_ll).")
        grid_ll = _grid_lookup_from_meta(Path(GRID_META_VIZ))
    if "ses_ll" not in globals():
        if "_build_ses_ll" not in globals():
            raise RuntimeError("Please run the previous population-distance cell first (it defines ses_ll).")
        ses_ll = _build_ses_ll()

    con.register("grid_ll", grid_ll)
    con.register("ses_ll", ses_ll)

    cache = OUT_CITY_DIR / f"pop_meal_dist_{CITY}_{window}_{is_weekend}.parquet"
    if cache.exists() and not WD_RECOMPUTE:
        return con.execute("select * from read_parquet(?)", [str(cache)]).fetchdf()

    bin_km = float(WD_BIN_KM)
    max_km = float(WD_MAX_KM)

    sql = f'''
    with base as (
      select
        uuid,
        regexp_extract(grid_uid, '.*_(\\d+)_\\d+$', 1)::int as col,
        regexp_extract(grid_uid, '.*_\\d+_(\\d+)$', 1)::int as row,
        sum(tau_filled_min) as w_sum
      from read_parquet(?)
      where "window" = ? and is_weekend = ?
      group by 1,2,3
    ),
    joined as (
      select
        b.w_sum,
        g.lon,
        g.lat,
        s.home_lon,
        s.home_lat,
        s.work_lon,
        s.work_lat
      from base b
      join grid_ll g using(col, row)
      join ses_ll s using(uuid)
    ),
    dist as (
      select
        'home' as place,
        6371.0 * 2 * asin(least(1.0, sqrt(
          pow(sin(radians(home_lat - lat) / 2), 2)
          + cos(radians(lat)) * cos(radians(home_lat))
          * pow(sin(radians(home_lon - lon) / 2), 2)
        ))) as dist_km,
        1::bigint as n_locs,
        w_sum as w_sum
      from joined
      where home_lon is not null and home_lat is not null

      union all

      select
        'work' as place,
        6371.0 * 2 * asin(least(1.0, sqrt(
          pow(sin(radians(work_lat - lat) / 2), 2)
          + cos(radians(lat)) * cos(radians(work_lat))
          * pow(sin(radians(work_lon - lon) / 2), 2)
        ))) as dist_km,
        1::bigint as n_locs,
        w_sum as w_sum
      from joined
      where work_lon is not null and work_lat is not null
    ),
    binned as (
      select
        place,
        floor(dist_km / {bin_km}) * {bin_km} as bin_km,
        sum(n_locs) as n_locs,
        sum(w_sum) as w_sum
      from dist
      where dist_km >= 0 and dist_km < {max_km}
      group by 1,2
    )
    select * from binned
    order by place, bin_km
    '''

    params: list[object] = [str(OUT_FILE), window, bool(is_weekend)]
    con.execute(f"COPY ({sql}) TO '{cache.as_posix()}' (FORMAT PARQUET)", params)
    return con.execute("select * from read_parquet(?)", [str(cache)]).fetchdf()


def _plot_work_home(bins_df: pd.DataFrame, *, title: str) -> go.Figure:
    df = bins_df.copy()
    df["bin_km"] = pd.to_numeric(df["bin_km"], errors="coerce")
    df["n_locs"] = pd.to_numeric(df["n_locs"], errors="coerce")
    df["w_sum"] = pd.to_numeric(df["w_sum"], errors="coerce")

    bin_km = float(WD_BIN_KM)
    max_km = float(WD_MAX_KM)
    bin_starts = np.arange(0.0, max_km, bin_km)
    centers = bin_starts + bin_km / 2

    val_col = "n_locs" if WD_WEIGHT_MODE == "location_share" else "w_sum"
    y_title = "Share of meal locations (%)" if WD_WEIGHT_MODE == "location_share" else "Share of meal exposure (%)"

    fig = make_subplots(
        rows=1,
        cols=2,
        shared_yaxes=True,
        subplot_titles=["Distance to work", "Distance to home"],
    )

    def add(place: str, col: int):
        s = df.loc[df["place"] == place].set_index("bin_km")[val_col]
        s = s.reindex(bin_starts, fill_value=0.0)
        share = s.to_numpy(dtype=float)
        share = share / share.sum() if share.sum() > 0 else share
        pct = 100.0 * share
        smooth = _gaussian_smooth_edge(pct, sigma_bins=float(WD_SMOOTH_SIGMA_BINS))

        fig.add_trace(
            go.Bar(
                x=centers,
                y=pct,
                width=bin_km,
                name="bin %",
                opacity=0.65,
                marker_color="rgba(76, 120, 168, 0.65)",
                showlegend=False,
            ),
            row=1,
            col=col,
        )
        fig.add_trace(
            go.Scatter(
                x=centers,
                y=smooth,
                mode="lines",
                name="smooth",
                line={"color": "black", "width": 2},
                showlegend=False,
            ),
            row=1,
            col=col,
        )

        # Highlight the short-distance range.
        fig.add_vrect(
            x0=0,
            x1=2.0,
            fillcolor="rgba(0,0,0,0.06)",
            line_width=0,
            row=1,
            col=col,
        )
        fig.add_vline(x=2.0, line_width=1, line_dash="dot", line_color="gray", row=1, col=col)

        # Highlight: cumulative share within 2km.
        within2 = float(pct[centers <= 2.0].sum())
        xref = "x domain" if col == 1 else f"x{col} domain"
        fig.add_annotation(
            x=0.98,
            y=0.98,
            xref=xref,
            yref="y domain",
            text=f"≤2 km: {within2:.1f}%",
            showarrow=False,
            font={"family": WD_FONT_FAMILY, "size": 12, "color": "black"},
            align="right",
        )

    add("work", 1)
    add("home", 2)

    fig.update_layout(
        title=title,
        height=420,
        width=1050,
        template="simple_white",
        font={"family": WD_FONT_FAMILY, "size": 14, "color": "black"},
        margin={"r": 10, "t": 70, "l": 10, "b": 10},
        title_x=0.5,
    )
    fig.update_xaxes(title_text="Distance (km)", range=[0, max_km], dtick=1, row=1, col=1)
    fig.update_xaxes(title_text="Distance (km)", range=[0, max_km], dtick=1, row=1, col=2)
    fig.update_yaxes(title_text=y_title, row=1, col=1)
    return fig


# 1) Weekday lunch
bins_wd_lunch = _ensure_pop_bins("lunch", False)
fig_wd_lunch = _plot_work_home(
    bins_wd_lunch,
    title=f"Weekday lunch: distance from meal locations to Work/Home | bin={WD_BIN_KM} km | max={WD_MAX_KM} km",
)

# 2) Weekday dinner
bins_wd_dinner = _ensure_pop_bins("dinner", False)
fig_wd_dinner = _plot_work_home(
    bins_wd_dinner,
    title=f"Weekday dinner: distance from meal locations to Work/Home | bin={WD_BIN_KM} km | max={WD_MAX_KM} km",
)

display(fig_wd_lunch)
_save_png(fig_wd_lunch, "weekday_lunch_dist_work_home")
display(fig_wd_dinner)
_save_png(fig_wd_dinner, "weekday_dinner_dist_work_home")


[font] Optima LT Pro | file exists: True


[OK] Saved: E:\OneDrive\HKU\Project\202512_EFE\data\expo\figures_meal_distance\weekday_lunch_dist_work_home_weight_share_bin0p5km_max15p0km.png


[OK] Saved: E:\OneDrive\HKU\Project\202512_EFE\data\expo\figures_meal_distance\weekday_dinner_dist_work_home_weight_share_bin0p5km_max15p0km.png


WindowsPath('E:/OneDrive/HKU/Project/202512_EFE/data/expo/figures_meal_distance/weekday_dinner_dist_work_home_weight_share_bin0p5km_max15p0km.png')

In [ ]:
# =========================================================
# Weekend lunch/dinner: meal-location distance to Work/Home
# (2 figures: lunch + dinner)
# =========================================================

# Reuse helpers/style from the previous cell.
_need = ["_ensure_pop_bins", "_plot_work_home", "_save_png"]
_missing = [n for n in _need if n not in globals()]
if _missing:
    raise RuntimeError(
        "Run the previous Weekday lunch/dinner cell first; missing: " + ", ".join(_missing)
    )

# 1) Weekend lunch
bins_we_lunch = _ensure_pop_bins("lunch", True)
fig_we_lunch = _plot_work_home(
    bins_we_lunch,
    title=f"Weekend lunch: distance from meal locations to Work/Home | bin={WD_BIN_KM} km | max={WD_MAX_KM} km",
)

# 2) Weekend dinner
bins_we_dinner = _ensure_pop_bins("dinner", True)
fig_we_dinner = _plot_work_home(
    bins_we_dinner,
    title=f"Weekend dinner: distance from meal locations to Work/Home | bin={WD_BIN_KM} km | max={WD_MAX_KM} km",
)

display(fig_we_lunch)
_save_png(fig_we_lunch, "weekend_lunch_dist_work_home")
display(fig_we_dinner)
_save_png(fig_we_dinner, "weekend_dinner_dist_work_home")


[OK] Saved: E:\OneDrive\HKU\Project\202512_EFE\data\expo\figures_meal_distance\weekend_lunch_dist_work_home_weight_share_bin0p5km_max15p0km.png


[OK] Saved: E:\OneDrive\HKU\Project\202512_EFE\data\expo\figures_meal_distance\weekend_dinner_dist_work_home_weight_share_bin0p5km_max15p0km.png


WindowsPath('E:/OneDrive/HKU/Project/202512_EFE/data/expo/figures_meal_distance/weekend_dinner_dist_work_home_weight_share_bin0p5km_max15p0km.png')

: 

In [15]:
pip install --upgrade kaleido

Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# =========================
# 0) 配置：改这里
# =========================
gpkg_path = r"E:/OneDrive/HKU/Project/202512_EFE/data/gpkg/Shenzhen_440300.gpkg"
layer_name = "grid_500m_access_k10_destination_v1"
out_dir = r"E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis"
os.makedirs(out_dir, exist_ok=True)

value_cols = ["S_off", "S_on", "S_all"]
weight_col = "demand_pop"   # 人口权重（可为0，但不要为负）
geometry_col = "geometry"

# =========================
# 1) 读数据 & 基础清洗
# =========================
gdf = gpd.read_file(gpkg_path, layer=layer_name)

# 保留需要的列
keep_cols = [c for c in [*value_cols, weight_col, geometry_col] if c in gdf.columns]
gdf = gdf[keep_cols].copy()

# 强制数值化
for c in value_cols + [weight_col]:
    if c in gdf.columns:
        gdf[c] = pd.to_numeric(gdf[c], errors="coerce")

# 处理权重：缺失->0；负值->报错
if weight_col not in gdf.columns:
    raise ValueError(f"Column '{weight_col}' not found in layer.")
gdf[weight_col] = gdf[weight_col].fillna(0)
if (gdf[weight_col] < 0).any():
    raise ValueError("demand_pop contains negative values, which is invalid for weights.")

# =========================
# 2) 指标函数：加权Gini / Theil / 等
# =========================
def _clean_xy(x, w=None):
    x = np.asarray(x, dtype=float)
    if w is None:
        mask = np.isfinite(x)
        x = x[mask]
        return x, None
    w = np.asarray(w, dtype=float)
    mask = np.isfinite(x) & np.isfinite(w) & (w >= 0)
    x = x[mask]
    w = w[mask]
    return x, w

def weighted_mean(x, w):
    s = w.sum()
    return np.nan if s == 0 else (x * w).sum() / s

def weighted_gini(x, w):
    """
    Population-weighted Gini coefficient
    Compatible with NumPy >= 2.0
    """
    x = np.asarray(x, dtype=float)
    w = np.asarray(w, dtype=float)

    mask = np.isfinite(x) & np.isfinite(w) & (w >= 0)
    x = x[mask]
    w = w[mask]

    if len(x) == 0 or w.sum() == 0:
        return np.nan

    order = np.argsort(x)
    x = x[order]
    w = w[order]

    W = np.cumsum(w)
    Y = np.cumsum(x * w)

    W_tot = W[-1]
    Y_tot = Y[-1]
    if Y_tot == 0:
        return 0.0

    p = np.insert(W / W_tot, 0, 0.0)
    L = np.insert(Y / Y_tot, 0, 0.0)

    # NumPy 2.0+ compatible
    area = np.trapezoid(L, p)
    return float(1 - 2 * area)


def theil_T(x, w=None):
    """
    Theil T（熵不均衡），可加权。
    x需非负且均值>0；若含<=0，会在清洗后剔除或返回NaN。
    """
    if w is None:
        x, _ = _clean_xy(x, None)
        x = x[np.isfinite(x)]
        x = x[x > 0]
        if len(x) == 0:
            return np.nan
        mu = x.mean()
        return float(np.mean((x / mu) * np.log(x / mu)))
    else:
        x, w = _clean_xy(x, w)
        mask = x > 0
        x = x[mask]
        w = w[mask]
        if len(x) == 0 or w.sum() == 0:
            return np.nan
        mu = weighted_mean(x, w)
        if not np.isfinite(mu) or mu <= 0:
            return np.nan
        p = w / w.sum()
        return float(np.sum(p * (x / mu) * np.log(x / mu)))

def coeff_var(x, w=None):
    """变异系数 CV = std/mean（可加权）"""
    if w is None:
        x, _ = _clean_xy(x, None)
        if len(x) == 0:
            return np.nan
        mu = np.mean(x)
        sd = np.std(x)
        return np.nan if mu == 0 else float(sd / mu)
    else:
        x, w = _clean_xy(x, w)
        if len(x) == 0 or w.sum() == 0:
            return np.nan
        mu = weighted_mean(x, w)
        if mu == 0 or not np.isfinite(mu):
            return np.nan
        p = w / w.sum()
        var = np.sum(p * (x - mu) ** 2)
        sd = np.sqrt(var)
        return float(sd / mu)

def weighted_quantile(x, q, w):
    """加权分位数 q in [0,1]"""
    x, w = _clean_xy(x, w)
    if len(x) == 0 or w.sum() == 0:
        return np.nan
    order = np.argsort(x)
    x = x[order]
    w = w[order]
    cw = np.cumsum(w)
    cutoff = q * w.sum()
    return float(np.interp(cutoff, cw, x))

def top_share(x, w=None, top_q=0.10):
    """
    Top (top_q) 占比：总量中由“最高 top_q 分位的单元”贡献的比例
    - unweighted: 按单元数分位
    - weighted: 按权重分位（更像“Top10%人口”所在格的供给占比）
    """
    if w is None:
        x, _ = _clean_xy(x, None)
        if len(x) == 0:
            return np.nan
        thr = np.nanquantile(x, 1 - top_q)
        total = np.nansum(x)
        return np.nan if total == 0 else float(np.nansum(x[x >= thr]) / total)
    else:
        x, w = _clean_xy(x, w)
        if len(x) == 0 or w.sum() == 0:
            return np.nan
        thr = weighted_quantile(x, 1 - top_q, w)
        total = np.sum(x * w)
        return np.nan if total == 0 else float(np.sum((x * w)[x >= thr]) / total)

# =========================
# 3) 汇总表：极化/均匀指标（含人口加权）
# =========================
rows = []
for col in value_cols:
    if col not in gdf.columns:
        print(f"[WARN] {col} not found, skip.")
        continue

    x = gdf[col].to_numpy()
    w = gdf[weight_col].to_numpy()

    # unweighted quantiles
    x_clean, _ = _clean_xy(x, None)
    p10 = np.nanquantile(x_clean, 0.10) if len(x_clean) else np.nan
    p50 = np.nanquantile(x_clean, 0.50) if len(x_clean) else np.nan
    p90 = np.nanquantile(x_clean, 0.90) if len(x_clean) else np.nan

    # weighted quantiles
    wp10 = weighted_quantile(x, 0.10, w)
    wp50 = weighted_quantile(x, 0.50, w)
    wp90 = weighted_quantile(x, 0.90, w)

    rows.append({
        "metric": col,
        "n_grids": int(np.isfinite(x).sum()),
        "sum_unweighted": float(np.nansum(x)),
        "mean_unweighted": float(np.nanmean(x)),
        "gini_unweighted": weighted_gini(x, np.ones_like(x, dtype=float)),
        "theilT_unweighted_pos": theil_T(x, None),
        "cv_unweighted": coeff_var(x, None),
        "p90_p10_unweighted": float(p90 / p10) if (np.isfinite(p90) and np.isfinite(p10) and p10 != 0) else np.nan,
        "top10_share_unweighted": top_share(x, None, 0.10),

        "pop_sum": float(np.nansum(w)),
        "mean_pop_weighted": weighted_mean(x, w),
        "gini_pop_weighted": weighted_gini(x, w),
        "theilT_pop_weighted_pos": theil_T(x, w),
        "cv_pop_weighted": coeff_var(x, w),
        "p90_p10_pop_weighted": float(wp90 / wp10) if (np.isfinite(wp90) and np.isfinite(wp10) and wp10 != 0) else np.nan,
        "top10_share_pop_weighted": top_share(x, w, 0.10),

        "p10_unweighted": p10, "p50_unweighted": p50, "p90_unweighted": p90,
        "p10_pop_weighted": wp10, "p50_pop_weighted": wp50, "p90_pop_weighted": wp90,
    })

summary = pd.DataFrame(rows)
summary_path = os.path.join(out_dir, "summary_metrics.csv")
summary.to_csv(summary_path, index=False, encoding="utf-8-sig")
print(f"[OK] Saved: {summary_path}")

# =========================
# 4) 分位分组表（deciles）：看“集中度/极化”更直观
# =========================
def decile_table(df, xcol, wcol, n=10):
    tmp = df[[xcol, wcol]].copy()
    tmp = tmp[np.isfinite(tmp[xcol])].copy()
    tmp[wcol] = tmp[wcol].fillna(0)

    # unweighted deciles
    tmp["decile_unw"] = pd.qcut(tmp[xcol], q=n, labels=False, duplicates="drop") + 1

    # 对每个decile：单元数、人口、均值、总供给(值*人口)、人口占比、供给占比
    agg = tmp.groupby("decile_unw").agg(
        n_grids=(xcol, "size"),
        pop=(wcol, "sum"),
        mean_x=(xcol, "mean"),
        sum_x=(xcol, "sum"),
        sum_x_pop=(xcol, lambda s: float(np.sum(s.to_numpy() * tmp.loc[s.index, wcol].to_numpy())))
    ).reset_index()

    total_pop = agg["pop"].sum()
    total_supply_pop = agg["sum_x_pop"].sum()

    agg["pop_share"] = agg["pop"] / total_pop if total_pop != 0 else np.nan
    agg["supply_share_pop"] = agg["sum_x_pop"] / total_supply_pop if total_supply_pop != 0 else np.nan
    return agg.sort_values("decile_unw")

for col in value_cols:
    if col not in gdf.columns:
        continue
    dec = decile_table(gdf, col, weight_col, n=10)
    dec_path = os.path.join(out_dir, f"deciles_{col}.csv")
    dec.to_csv(dec_path, index=False, encoding="utf-8-sig")
    print(f"[OK] Saved: {dec_path}")

# =========================
# 5) 图 1：直方图（log可选）
# =========================
for col in value_cols:
    if col not in gdf.columns:
        continue
    x = gdf[col].to_numpy()
    x = x[np.isfinite(x)]

    plt.figure()
    plt.hist(x, bins=60)  # 默认颜色
    plt.title(f"Histogram of {col}")
    plt.xlabel(col)
    plt.ylabel("Count")
    hist_path = os.path.join(out_dir, f"hist_{col}.png")
    plt.tight_layout()
    plt.savefig(hist_path, dpi=200)
    plt.close()
    print(f"[OK] Saved: {hist_path}")

# =========================
# 6) 图 2：Lorenz 曲线（人口加权 & 非加权）
# =========================
def lorenz_curve(x, w=None):
    if w is None:
        x, _ = _clean_xy(x, None)
        if len(x) == 0:
            return None, None
        w = np.ones_like(x, dtype=float)
    else:
        x, w = _clean_xy(x, w)
        if len(x) == 0 or w.sum() == 0:
            return None, None

    order = np.argsort(x)
    x = x[order]; w = w[order]
    W = np.cumsum(w); Y = np.cumsum(x * w)
    W_tot = W[-1]; Y_tot = Y[-1]
    if Y_tot == 0:
        p = np.insert(W / W_tot, 0, 0.0)
        L = np.insert(np.zeros_like(W, dtype=float), 0, 0.0)
        return p, L
    p = np.insert(W / W_tot, 0, 0.0)
    L = np.insert(Y / Y_tot, 0, 0.0)
    return p, L

for col in value_cols:
    if col not in gdf.columns:
        continue
    x = gdf[col].to_numpy()
    w = gdf[weight_col].to_numpy()

    p1, L1 = lorenz_curve(x, None)
    p2, L2 = lorenz_curve(x, w)

    plt.figure()
    if p1 is not None:
        plt.plot(p1, L1, label="Unweighted")
    if p2 is not None:
        plt.plot(p2, L2, label="Pop-weighted")
    # 平等线
    plt.plot([0, 1], [0, 1], linestyle="--", label="Equality")
    plt.title(f"Lorenz Curve: {col}")
    plt.xlabel("Cumulative share of units (or population)")
    plt.ylabel("Cumulative share of supply intensity (x * weight)")
    plt.legend()
    lorenz_path = os.path.join(out_dir, f"lorenz_{col}.png")
    plt.tight_layout()
    plt.savefig(lorenz_path, dpi=200)
    plt.close()
    print(f"[OK] Saved: {lorenz_path}")

# =========================
# 7) 图 3：空间分布图（按分位数分级）
# =========================
for col in value_cols:
    if col not in gdf.columns:
        continue

    gtmp = gdf[[col, geometry_col]].copy()
    gtmp = gtmp[np.isfinite(gtmp[col])].copy()
    if len(gtmp) == 0:
        continue

    # 分位数分级（10档）
    try:
        gtmp["q10"] = pd.qcut(gtmp[col], 10, labels=False, duplicates="drop") + 1
    except ValueError:
        # 如果值太离散/重复导致qcut失败，就退回5档
        gtmp["q10"] = pd.qcut(gtmp[col], 5, labels=False, duplicates="drop") + 1

    plt.figure()
    ax = plt.gca()
    gtmp.plot(column="q10", legend=True, ax=ax)  # 默认配色
    ax.set_title(f"Quantile map of {col} (higher=stronger)")
    ax.set_axis_off()
    map_path = os.path.join(out_dir, f"map_quantile_{col}.png")
    plt.tight_layout()
    plt.savefig(map_path, dpi=250)
    plt.close()
    print(f"[OK] Saved: {map_path}")

print("\nAll done.")
print("Outputs in:", out_dir)

[OK] Saved: E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\summary_metrics.csv
[OK] Saved: E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\deciles_S_off.csv
[OK] Saved: E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\deciles_S_on.csv
[OK] Saved: E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\deciles_S_all.csv
[OK] Saved: E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\hist_S_off.png
[OK] Saved: E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\hist_S_on.png
[OK] Saved: E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\hist_S_all.png
[OK] Saved: E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\lorenz_S_off.png
[OK] Saved: E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\lorenz_S_on.png
[OK] Saved: E:/OneDriv

In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# 你已有：gdf, out_dir, value_cols, weight_col
# 确保至少有 S_on, S_off
pairs = [("S_off", "S_on"), ("S_all", None)]  # 第二个 None 表示单独画 S_all（可选）

def _clean_xy(x, w=None):
    x = np.asarray(x, dtype=float)
    if w is None:
        m = np.isfinite(x)
        return x[m], None
    w = np.asarray(w, dtype=float)
    m = np.isfinite(x) & np.isfinite(w) & (w >= 0)
    return x[m], w[m]

def weighted_quantile(x, q, w):
    x, w = _clean_xy(x, w)
    if len(x) == 0 or w.sum() == 0:
        return np.nan
    order = np.argsort(x)
    x = x[order]; w = w[order]
    cw = np.cumsum(w)
    cutoff = q * w.sum()
    return float(np.interp(cutoff, cw, x))

def lorenz_curve(x, w=None):
    if w is None:
        x, _ = _clean_xy(x, None)
        if len(x) == 0:
            return None, None
        w = np.ones_like(x, dtype=float)
    else:
        x, w = _clean_xy(x, w)
        if len(x) == 0 or w.sum() == 0:
            return None, None

    order = np.argsort(x)
    x = x[order]; w = w[order]
    W = np.cumsum(w)
    Y = np.cumsum(x * w)
    W_tot = W[-1]
    Y_tot = Y[-1]
    p = np.insert(W / W_tot, 0, 0.0)
    if Y_tot == 0:
        L = np.insert(np.zeros_like(Y, dtype=float), 0, 0.0)
    else:
        L = np.insert(Y / Y_tot, 0, 0.0)
    return p, L

def _plot_hist_compare(ax, gdf, col_a, col_b=None, bins=80, log1p=True, title=None):
    xa = gdf[col_a].to_numpy()
    xa = xa[np.isfinite(xa)]
    if log1p:
        xa = np.log1p(np.clip(xa, a_min=0, a_max=None))

    ax.hist(xa, bins=bins, density=True, alpha=0.45, label=col_a)

    if col_b is not None:
        xb = gdf[col_b].to_numpy()
        xb = xb[np.isfinite(xb)]
        if log1p:
            xb = np.log1p(np.clip(xb, a_min=0, a_max=None))
        ax.hist(xb, bins=bins, density=True, alpha=0.45, label=col_b)

    ax.set_title(title or "Distribution comparison")
    ax.set_xlabel("log1p(S) (if log1p=True) else S")
    ax.set_ylabel("Density")
    ax.legend()

def _plot_lorenz_compare(ax, gdf, col_a, col_b=None, wcol=None, title=None):
    xa = gdf[col_a].to_numpy()
    wa = gdf[wcol].to_numpy() if wcol else None

    p, L = lorenz_curve(xa, wa)
    if p is not None:
        ax.plot(p, L, label=f"{col_a} ({'pop-w' if wcol else 'unw'})")

    if col_b is not None:
        xb = gdf[col_b].to_numpy()
        wb = gdf[wcol].to_numpy() if wcol else None
        p2, L2 = lorenz_curve(xb, wb)
        if p2 is not None:
            ax.plot(p2, L2, label=f"{col_b} ({'pop-w' if wcol else 'unw'})")

    ax.plot([0, 1], [0, 1], linestyle="--", label="Equality")
    ax.set_title(title or "Lorenz comparison")
    ax.set_xlabel("Cumulative share (population if weighted)")
    ax.set_ylabel("Cumulative share of supply (x * weight)")
    ax.legend()

def _plot_map_quantiles_side_by_side(gdf, col_a, col_b, out_ax_a, out_ax_b, q=10):
    # 分位数分级，重复值多时会降档
    def assign_q(s, q):
        s = s.copy()
        s = s[np.isfinite(s)]
        if len(s) == 0:
            return None
        try:
            return pd.qcut(s, q, labels=False, duplicates="drop") + 1
        except Exception:
            return pd.qcut(s, 5, labels=False, duplicates="drop") + 1

    import pandas as pd

    ga = gdf[[col_a, "geometry"]].copy()
    ga = ga[np.isfinite(ga[col_a])].copy()
    gb = gdf[[col_b, "geometry"]].copy()
    gb = gb[np.isfinite(gb[col_b])].copy()

    if len(ga) > 0:
        ga["q"] = pd.qcut(ga[col_a], q, labels=False, duplicates="drop") + 1
        ga.plot(column="q", legend=True, ax=out_ax_a)
        out_ax_a.set_title(f"{col_a} quantiles")
        out_ax_a.set_axis_off()

    if len(gb) > 0:
        gb["q"] = pd.qcut(gb[col_b], q, labels=False, duplicates="drop") + 1
        gb.plot(column="q", legend=True, ax=out_ax_b)
        out_ax_b.set_title(f"{col_b} quantiles")
        out_ax_b.set_axis_off()

# =========================
# 生成 PDF 报告（多页）
# =========================
pdf_path = os.path.join(out_dir, "compare_online_offline.pdf")

# 简单 sanity check
for c in ["S_off", "S_on", "S_all", weight_col]:
    if c not in gdf.columns:
        print(f"[WARN] missing column: {c}")

with PdfPages(pdf_path) as pdf:
    # Page 1: Histogram compare (unweighted, log1p)
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    _plot_hist_compare(
        ax, gdf, "S_off", "S_on",
        bins=80, log1p=True,
        title="Online vs Offline supply intensity distribution (log1p scaled)"
    )
    fig.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

    # Page 2: Lorenz compare (unweighted)
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    _plot_lorenz_compare(
        ax, gdf, "S_off", "S_on",
        wcol=None,
        title="Lorenz curve: Online vs Offline (unweighted)"
    )
    fig.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

    # Page 3: Lorenz compare (population-weighted)
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    _plot_lorenz_compare(
        ax, gdf, "S_off", "S_on",
        wcol=weight_col,
        title="Lorenz curve: Online vs Offline (population-weighted by demand_pop)"
    )
    fig.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

    # Page 4: Maps side-by-side (quantile)
    # 如果你数据量很大，画图会慢；可以先 gdf = gdf.sample(...) 或 dissolve/clip 再画
    try:
        fig, axes = plt.subplots(1, 2, figsize=(14, 7))
        _plot_map_quantiles_side_by_side(gdf, "S_off", "S_on", axes[0], axes[1], q=10)
        fig.suptitle("Spatial distribution (quantile classes): Offline vs Online", y=0.98)
        fig.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)
    except Exception as e:
        print("[WARN] Map page skipped due to:", repr(e))

    # Page 5 (optional): S_all distribution + Lorenz
    if "S_all" in gdf.columns:
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        _plot_hist_compare(ax, gdf, "S_all", None, bins=80, log1p=True,
                           title="Overall supply intensity distribution (S_all, log1p scaled)")
        fig.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        _plot_lorenz_compare(ax, gdf, "S_all", None, wcol=weight_col,
                             title="Lorenz curve: S_all (population-weighted)")
        fig.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

print(f"[OK] Saved PDF report: {pdf_path}")


[OK] Saved PDF report: E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\compare_online_offline.pdf


In [18]:
# =========================================================
# Font setup: Optima LT Pro (matplotlib)
# =========================================================
import matplotlib as mpl
import matplotlib.font_manager as fm

# 1️⃣ 如果系统已安装 Optima LT Pro（macOS / 部分 Windows）
mpl.rcParams["font.family"] = "Optima LT Pro"

# 2️⃣ 保险起见：明确指定字体文件（推荐）
# 把路径换成你机器上的真实路径
font_path = "E:\\OneDrive\\HKU\Project\\202512_EFE\\OptimaLTPro-Roman.otf"   # or .ttf
fm.fontManager.addfont(font_path)
    
optima = fm.FontProperties(fname=font_path)
mpl.rcParams["font.family"] = optima.get_name()

# 3️⃣ 数学/符号也尽量统一风格
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = optima.get_name()
mpl.rcParams["mathtext.it"] = optima.get_name()
mpl.rcParams["mathtext.bf"] = optima.get_name()


<>:12: SyntaxWarning:

"\P" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\P"? A raw string is also an option.

<>:12: SyntaxWarning:

"\P" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\P"? A raw string is also an option.

C:\Users\weipd\AppData\Local\Temp\ipykernel_29184\386311867.py:12: SyntaxWarning:

"\P" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\P"? A raw string is also an option.



In [19]:
# =========================================================
# Extra cell: Online vs Offline Lorenz (same PNG, transparent)
# =========================================================
import os
import numpy as np
import matplotlib.pyplot as plt

# 取数据
x_off = gdf["RQ_off"].to_numpy()
x_on  = gdf["RQ_on"].to_numpy()
w     = gdf[weight_col].to_numpy()

# 计算 Lorenz（人口加权；如要非加权，把 w 换成 None）
p_off, L_off = lorenz_curve(x_off, w)
p_on,  L_on  = lorenz_curve(x_on,  w)

# 画图（同一张图）
fig, ax = plt.subplots(figsize=(6, 6))

ax.plot(p_off, L_off, label="Offline (RQ_off)", linewidth=2)
ax.plot(p_on,  L_on,  label="Online (RQ_on)",  linewidth=2)
ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1, label="Equality")

ax.set_xlabel("Cumulative population share")
ax.set_ylabel("Cumulative supply share")
ax.set_title("Lorenz curve: Online vs Offline (population-weighted)")
ax.legend()

# 关键点：透明背景
fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

# 保存 PNG（透明）
png_path = os.path.join(out_dir, "lorenz_RQ_online_vs_offline.png")
plt.savefig(png_path, dpi=300, transparent=True, bbox_inches="tight")
plt.close()

print(f"[OK] Saved: {png_path}")


KeyError: 'RQ_off'

In [ ]:
# =========================================================
# Lorenz curves: RQ_off vs RQ_on vs delta_RQ (same PNG, transparent)
# =========================================================
import os
import numpy as np
import matplotlib.pyplot as plt
# =========================
# 0) 配置：改这里
# =========================
gpkg_path = r"E:/OneDrive/HKU/Project/202512_EFE/data/gpkg/Shenzhen_440300.gpkg"
layer_name = "grid_500m_access_k10_destination_v1"
out_dir = r"E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis"
os.makedirs(out_dir, exist_ok=True)

value_cols = ["RQ_off", "RQ_on", "delta_RQ"]
weight_col = "demand_pop"   # 人口权重（可为0，但不要为负）
geometry_col = "geometry"

# =========================
# 1) 读数据 & 基础清洗
# =========================
gdf = gpd.read_file(gpkg_path, layer=layer_name)

# 保留需要的列
keep_cols = [c for c in [*value_cols, weight_col, geometry_col] if c in gdf.columns]
gdf = gdf[keep_cols].copy()
# --- config ---
cols = ["RQ_off", "RQ_on"]
weight_col = "demand_pop"   # population weight
png_name = "lorenz_RQ_off_RQ_on_deltaRQ_pop_weighted.png"

# --- sanity check ---
missing = [c for c in cols + [weight_col] if c not in gdf.columns]
if missing:
    raise ValueError(f"Missing columns in gdf: {missing}")

w = gdf[weight_col].to_numpy()

# --- compute Lorenz curves (population-weighted) ---
curves = {}
for c in cols:
    x = gdf[c].to_numpy()
    p, L = lorenz_curve(x, w)   # if you want unweighted: lorenz_curve(x, None)
    if p is None or L is None:
        print(f"[WARN] skipped {c} (empty/invalid after cleaning)")
        continue
    curves[c] = (p, L)

# --- plot ---
fig, ax = plt.subplots(figsize=(6.2, 6.2))

for c, (p, L) in curves.items():
    ax.plot(p, L, linewidth=2, label=c)

ax.plot([0, 1], [0, 1], linestyle="--", linewidth=1, label="Equality")

ax.set_xlabel("Cumulative population share")
ax.set_ylabel("Cumulative nutrition-quality exposure share (RQ × population)")
ax.set_title("Lorenz curves: RQ_off vs RQ_on (population-weighted)")
ax.legend()

# transparent background
fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

# save
png_path = os.path.join(out_dir, png_name)
plt.savefig(png_path, dpi=300, transparent=True, bbox_inches="tight")
plt.close()

print(f"[OK] Saved: {png_path}")


[OK] Saved: E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\lorenz_RQ_off_RQ_on_deltaRQ_pop_weighted.png


In [ ]:
# =========================================================
# Line plot 3: Spatial (radial) profile of ΔRQ vs distance to city center
# =========================================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1) 选一个中心点：这里用全体网格的几何中心（也可换成CBD点）
gdf_proj = gdf.to_crs(gdf.estimate_utm_crs())  # 投影到米制
center = gdf_proj.geometry.unary_union.centroid

# 2) 距离（米）
dist = gdf_proj.geometry.centroid.distance(center).to_numpy()

delta = gdf_proj["delta_RQ"].to_numpy(float)
w = gdf_proj["demand_pop"].to_numpy(float)

m = np.isfinite(dist) & np.isfinite(delta) & np.isfinite(w) & (w >= 0)
dist = dist[m]; delta = delta[m]; w = w[m]

# 3) 分箱（比如 30 个距离段）
bins = 30
bin_id = pd.qcut(dist, bins, labels=False, duplicates="drop")

df = pd.DataFrame({"bin": bin_id, "dist": dist, "delta": delta, "w": w})
grp = df.groupby("bin")

def wmean(x, w):
    s = w.sum()
    return np.nan if s == 0 else (x*w).sum()/s

profile = grp.apply(lambda g: pd.Series({
    "dist_mid": np.median(g["dist"]),
    "delta_wmean": wmean(g["delta"].to_numpy(), g["w"].to_numpy()),
    "pop": g["w"].sum()
})).reset_index(drop=True).sort_values("dist_mid")

# 4) 画线图
fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(profile["dist_mid"]/1000, profile["delta_wmean"], linewidth=2)

ax.axhline(0, linestyle="--", linewidth=1)
ax.set_xlabel("Distance to center (km)")
ax.set_ylabel("Population-weighted mean ΔRQ")
ax.set_title("Spatial profile of ΔRQ along center-periphery gradient")

fig.patch.set_alpha(0); ax.patch.set_alpha(0)
out = os.path.join(out_dir, "spatial_profile_delta_RQ_distance.png")
plt.savefig(out, dpi=300, transparent=True, bbox_inches="tight")
plt.close()
print("[OK]", out)


C:\Users\weipd\AppData\Local\Temp\ipykernel_49192\4180064272.py:11: DeprecationWarning:

The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.



[OK] E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\spatial_profile_delta_RQ_distance.png


C:\Users\weipd\AppData\Local\Temp\ipykernel_49192\4180064272.py:33: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [ ]:
# =========================================================
# Line plot 1: Population-weighted CDF of delta_RQ
# =========================================================
import os
import numpy as np
import matplotlib.pyplot as plt

delta = gdf["delta_RQ"].to_numpy(dtype=float)
w = gdf["demand_pop"].to_numpy(dtype=float)

mask = np.isfinite(delta) & np.isfinite(w) & (w >= 0)
delta = delta[mask]
w = w[mask]

order = np.argsort(delta)
delta_s = delta[order]
w_s = w[order]

cw = np.cumsum(w_s)
cdf = cw / cw[-1] if cw[-1] > 0 else np.full_like(cw, np.nan)

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(delta_s, cdf, linewidth=2, label="Pop-weighted CDF")

ax.axvline(0, linestyle="--", linewidth=1, label="No change (ΔRQ=0)")
ax.set_xlabel("ΔRQ")
ax.set_ylabel("Cumulative population share")
ax.set_title("Population-weighted CDF of ΔRQ")
ax.legend()

fig.patch.set_alpha(0); ax.patch.set_alpha(0)
out = os.path.join(out_dir, "cdf_delta_RQ_pop_weighted.png")
plt.savefig(out, dpi=300, transparent=True, bbox_inches="tight")
plt.close()
print("[OK]", out)


[OK] E:/OneDrive/HKU/Project/202512_EFE/data/outputs/shenzhen_accessibility_analysis\cdf_delta_RQ_pop_weighted.png
